- 方法相同、拟合参数表达不同 ：  模拟拟合参数不同，EQ13
- 回归方程 ：要加时间限制，否则结果不一样；
- 函数中自动跳过为空的数据 ；联立方程可以定义每个方程的数据源？ 
- 单一方程式 ：预测与excel有差异 ，excel预测值是“联立方程的预测结果”？
- 单一方程参数 = 联立方程参数 ？ 基线拟合参数 = 场景拟合参数
- d 哑变量，是否作为数据源不变，还是需要提取规则，当其它发生变化，可能引起d变量发生改变

In [2]:
#引包：引入所需python包
import xlrd
import os
import re
import pandas as pd
import numpy as np
import statsmodels.formula.api as sm

from sklearn.metrics import roc_curve,auc
import matplotlib as mpl
from matplotlib import pyplot as plt
from numpy import nan
from sklearn import linear_model, datasets
from sklearn.model_selection import train_test_split
from statsmodels.tsa.statespace.sarimax import SARIMAX
import seaborn as sns
import time

from scipy.stats import kstest
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

In [3]:
dir = r"./data/"
print(os.listdir(dir))  #手术种类

['【20200703】-预测误差(2015-2017).xls', '【20200703】-模拟（2016，10）-3.xls', '【20200703】-模拟（2016，6-8）-2.xls', '【20200703】-模拟（2016,1-4）-1.xls', '其它数据.csv', '.ipynb_checkpoints', 'alleviewsdata.xlsx', '方程描述.csv']


# 数据源

## excel数据源
- 通过"预测误差表"获得数据源；

- 也可以通过模拟误差获得数据源，但是那样更复杂一些。

In [4]:
file = "alleviewsdata.xlsx"
df_ev= pd.read_excel(dir+file,"Sheet1",index_col = 0 ,na_values=['9999'])
df_ev.columns = df_ev.columns.str.lower()
df_ev.columns

Index(['con_fn_bj', 'con_fn_bj_0', 'con_fn_bj_1', 'con_fn_bj_10',
       'con_fn_bj_2', 'con_fn_bj_3', 'con_fn_bj_4', 'con_fn_bj_5',
       'con_fn_bj_6', 'con_fn_bj_7',
       ...
       'yht_un_tj_1', 'yht_un_tj_10', 'yht_un_tj_2', 'yht_un_tj_3',
       'yht_un_tj_4', 'yht_un_tj_5', 'yht_un_tj_6', 'yht_un_tj_7',
       'yht_un_tj_8', 'yht_un_tj_9'],
      dtype='object', length=3722)

## 衍生变量
- 根据定义式衍生的变量

# 定义
## 参数
## 函数

# 天津
## Eq 100 : gdp1r_tj 行为方程 no 结果有差异
行为方程 ： gdp1r_tj  = cul_area_tj + gdp1r_tj(-1)

In [11]:
test1 = df_ev[["gdp1r_tj","cul_area_tj"]]
test1["gdplr_tj_s1"] = test1["gdp1r_tj"].shift(1)  ##直接写进表达式，结果有差异
test1 = test1[(test1.index>1996) & (test1.index <2018)]
results = sm.ols(formula = "gdp1r_tj ~ cul_area_tj  + gdplr_tj_s1 ",data = test1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               gdp1r_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     4584.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           4.26e-25
Time:                        07:51:58   Log-Likelihood:                -35.029
No. Observations:                  21   AIC:                             76.06
Df Residuals:                      18   BIC:                             79.19
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept       4.1284      1.297      3.184      0.005       1.404       6.852
cul_area_tj     0.0475      0.025      1.928      0.070      -0.004       0.099
gdplr_tj_s1     0.9907      0.012     81.846      0.000       0.965       1.016
==============================================================================
Omnibus:                        5.357   Durbin-Watson:                   1.157
Prob(Omnibus):                  0.069   Jarque-Bera (JB):                3.514
Skew:                          -0.980   Prob(JB):                        0.173
Kurtosis:                       3.422   Cond. No.                         550.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [114]:
results.predict()

array([126.82820558, 129.03194867, 130.07270933, 133.38891309,
       137.72502094, 141.37904066, 142.38851322, 141.8252671 ,
       138.50468726, 138.98539273, 139.88213844, 142.42550053,
       150.03792225, 147.17252221, 148.18577519, 152.51597548,
       156.50725341, 158.46014552, 139.83986369, 128.98768591,
       119.96448788])

## Eq101: gdpn_tj 定义方程 ok
定义方程 ： gdpn_tj  = gdp1n_tj  + gdp2n_tj  + gdp3n_tj

In [12]:
test1 = df_ev[["gdpn_tj","gdp1n_tj","gdp2n_tj","gdp3n_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp1n_tj"] + test1["gdp2n_tj"] + test1["gdp3n_tj"]
test1.head(2)

,gdpn_tj,gdp1n_tj,gdp2n_tj,gdp3n_tj,预测
1997,1264.63,69.52,676.01,519.10,1264.63
1998,1374.60,74.14,697.99,602.47,1374.60


In [ ]:
        if (y in exogs) & (len([x for x in x_list if x in exogs])==0):   
            
            preds = pd.merge(preds,test1[[y]],how = "left",left_index =True ,right_index = True)
            test_data[y] = preds[y]
            exogs.remove(y)
            Eqs.remove(Eqid)
            print(Eqid ,y,"----------------   求解成功 。")
        else:
            print(Eqid ,y, "还不能求解 ，因还未获得外生变量:  ",[x for x in x_list if x in exogs] )

## Eq102 ：gdpr_tj 定义方程 ok
定义方程 ： gdpr_tj  = gdp1r_tj  + gdp2r_tj  + gdp3r_tj

In [13]:
test1 = df_ev[["gdpr_tj","gdp1r_tj","gdp2r_tj","gdp3r_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp1r_tj"] + test1["gdp2r_tj"] + test1["gdp3r_tj"]
test1.head(2)

,gdpr_tj,gdp1r_tj,gdp2r_tj,gdp3r_tj,预测
1997,1706.422913,79.339985,627.011370,1000.071558,1706.422913
1998,1865.342166,83.624822,665.212326,1116.505019,1865.342166


## Eq103 : gdp1n_tj 定义方程 ok
定义方程 ： gdp1n_tj  = gdp1r_tj  * gdp1d_tj  / 100

In [14]:
test1 = df_ev[["gdp1n_tj","gdp1r_tj","gdp1d_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp1r_tj"]*test1["gdp1d_tj"]/100
test1.head(2)

,gdp1n_tj,gdp1r_tj,gdp1d_tj,验证结果
1997,69.52,79.339985,87.622905,69.52
1998,74.14,83.624822,88.657887,74.14


## Eq104 : gdp2_indan_tj 定义方程 ok
定义方程 ： gdp2_indan_tj  = gdp2_indar_tj  * gdp2_indad_tj  / 100


In [15]:
test1 = df_ev[["gdp2_indan_tj","gdp2_indar_tj","gdp2_indad_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["验证结果"] = test1["gdp2_indar_tj"]*test1["gdp2_indad_tj"]/100
test1.head(2)

,gdp2_indan_tj,gdp2_indar_tj,gdp2_indad_tj,验证结果
1997,609.65,552.721132,110.299745,609.65
1998,622.15,591.437786,105.192806,622.15


## Eq105：gdp2_bindan_tj 定义方程 ok
定义方程 ： gdp2_bindan_tj  = gdp2_bindar_tj  * gdp2_bindad_tj  / 100

In [16]:
test1 = df_ev[["gdp2_bindan_tj","gdp2_bindar_tj","gdp2_bindad_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp2_bindar_tj"]*test1["gdp2_bindad_tj"]/100
test1.head(2)

,gdp2_bindan_tj,gdp2_bindar_tj,gdp2_bindad_tj,预测
1997,66.36,140.365007,47.276740,66.36
1998,75.84,158.180566,47.945207,75.84


## Eq106 : gdp2n_tj 定义方程 ok
定义方程 ： gdp2n_tj  = gdp2r_tj  * gdp2d_tj  / 100


In [17]:
test1 = df_ev[["gdp2n_tj","gdp2r_tj","gdp2d_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp2r_tj"]*test1["gdp2d_tj"]/100
test1.head(2)

,gdp2n_tj,gdp2r_tj,gdp2d_tj,预测
1997,676.01,627.011370,107.814632,676.01
1998,697.99,665.212326,104.927400,697.99


## Eq107 ： gdp2r_tj2 定义方程 ok
定义方程 ： gdp2r_tj2  = gdp2_indar_tj  + gdp2_bindar_tj  + gdp2r_erro_tj

In [18]:
test1 = df_ev[["gdp2r_tj2","gdp2_indar_tj","gdp2_bindar_tj","gdp2r_erro_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = test1["gdp2_indar_tj"] + test1["gdp2_bindar_tj"] + test1["gdp2r_erro_tj"]
test1.head(2)

,gdp2r_tj2,gdp2_indar_tj,gdp2_bindar_tj,gdp2r_erro_tj,预测
1997,627.011370,552.721132,140.365007,-66.074770,627.011370
1998,665.212326,591.437786,158.180566,-84.406026,665.212326


## Eq108 : gdp2r_tj1 行为方程 ok
行为方程 ： gdp2r_tj1  = @coef(1)  + @coef(2) * gdp_tecn_tj / gdp2d_tj  + @coef(3) * gdp2r_tj1(-1)  + @coef(4) * d_gdp2r_tj_1

- gdp_tecn_tj / gdp2d_tj : 可以将表达式写入方程中，但参数估计结果不一样

In [26]:
test1 = df_ev[["gdp2r_tj1","gdp2_tecn_tj","gdp2d_tj","d_gdp2r_tj_1"]]
test1["gdp2r_tj1_s1"] = test1["gdp2r_tj1"].shift(1)
test1["gdp2_tecn_gdp2d"] = test1["gdp2_tecn_tj"]/test1["gdp2d_tj"]
test1 = test1[(test1.index>2013) & (test1.index <2020)]
test1.head(2)

,gdp2r_tj1,gdp2_tecn_tj,gdp2d_tj,d_gdp2r_tj_1,gdp2r_tj1_s1,gdp2_tecn_gdp2d
2014,7250.388989,971.870,109.422129,0.0,6595.951787,8.881841
2015,7967.070000,1148.218,100.000000,0.6,7250.388989,11.482180


In [27]:
#results = sm.ols(formula = "gdp2r_tj1 ~ gdp_tecn_tj/gdp2d_tj  + gdp2r_tj1.shift(1) + d_gdp2r_tj_1",data = test1 ).fit()
results = sm.ols(formula = "gdp2r_tj1 ~ gdp2_tecn_gdp2d  + gdp2r_tj1_s1 + d_gdp2r_tj_1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              gdp2r_tj1   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     400.3
Date:                Fri, 10 Jul 2020   Prob (F-statistic):            0.00249
Time:                        08:12:10   Log-Likelihood:                -27.848
No. Observations:                   6   AIC:                             63.70
Df Residuals:                       2   BIC:                             62.86
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept        3203.5309    206.382     15.522      0.004    2315.539    4091.523
gdp2_tecn_gdp2d    32.9102      5.323      6.183      0.025      10.007      55.813
gdp2r_tj1_s1        0.5709      0.033     17.441      0.003       0.430       0.712
d_gdp2r_tj_1      356.2237     45.762      7.784      0.016     159.324     553.123
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.592
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.262
Skew:                          -0.197   Prob(JB):                        0.877
Kurtosis:                       2.055   Cond. No.                     9.35e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.35e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq109 ：gdp2r_tj 定义方程  ok
定义方程 ： gdp2r_tj  = (gdp2r_tj1  + gdp2r_tj2)  / 2

In [28]:
test1 = df_ev[["gdp2r_tj","gdp2r_tj1","gdp2r_tj2"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = (test1["gdp2r_tj1"] + test1["gdp2r_tj2"])/2
test1.head(2)

,gdp2r_tj,gdp2r_tj1,gdp2r_tj2,预测
1997,627.011370,627.011370,627.011370,627.011370
1998,665.212326,665.212326,665.212326,665.212326


## Eq110 :gdp3n_tj 定义方程 ok
定义方程 ： gdp3n_tj  = gdp3r_tj  * gdp3d_tj  / 100

In [30]:
test1 = df_ev[["gdp3n_tj","gdp3r_tj","gdp3d_tj"]]
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1["预测"] = (test1["gdp3r_tj"] * test1["gdp3d_tj"])/100
test1.head(2)

,gdp3n_tj,gdp3r_tj,gdp3d_tj,预测
1997,519.10,1000.071558,51.906286,519.10
1998,602.47,1116.505019,53.960349,602.47


## Eq111 : gdp2_indar_tj 行为方程 ok
log(gdp2_indar_tj)  = @coef(1)  + @coef(2) * log(consr_tj)  + @coef(3) * log(investr_tj)  + @coef(4) * log(exusd_tj * exrate_cn)  + @coef(5) * d_gdp2_indar_tj

In [35]:
test1 = df_ev[["gdp2_indar_tj","consr_tj","investr_tj","exusd_tj","exrate_cn","d_gdp2_indar_tj"]]
test1["investr_tj_s1"] = test1["investr_tj"].shift(1)
test1 = test1[(test1.index>2000) & (test1.index <2019)]
test1.head(2)

,gdp2_indar_tj,consr_tj,investr_tj,exusd_tj,exrate_cn,d_gdp2_indar_tj,investr_tj_s1
2001,842.059957,1143.564338,910.766452,95.02,827.7,0.0,785.415921
2002,965.080761,1297.981315,1051.641182,115.95,827.7,-0.5,910.766452


In [36]:
results = sm.ols(formula = "np.log(gdp2_indar_tj) ~ np.log(consr_tj)  + np.log(investr_tj_s1) + np.log(exusd_tj * exrate_cn) + d_gdp2_indar_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(gdp2_indar_tj)   R-squared:                       0.999
Model:                               OLS   Adj. R-squared:                  0.999
Method:                    Least Squares   F-statistic:                     6494.
Date:                   Fri, 10 Jul 2020   Prob (F-statistic):           2.63e-21
Time:                           08:18:26   Log-Likelihood:                 47.855
No. Observations:                     18   AIC:                            -85.71
Df Residuals:                         13   BIC:                            -81.26
Df Model:                              4                                         
Covariance Type:               nonrobust                                         
================================================================================================
                                   coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------
Intercept                       -3.2657      0.437     -7.477      0.000      -4.209      -2.322
np.log(consr_tj)                 0.8262      0.113      7.294      0.000       0.582       1.071
np.log(investr_tj_s1)            0.1977      0.068      2.899      0.012       0.050       0.345
np.log(exusd_tj * exrate_cn)     0.2540      0.020     12.840      0.000       0.211       0.297
d_gdp2_indar_tj                  0.0666      0.007      9.558      0.000       0.052       0.082
==============================================================================
Omnibus:                        1.313   Durbin-Watson:                   1.869
Prob(Omnibus):                  0.519   Jarque-Bera (JB):                0.194
Skew:                          -0.087   Prob(JB):                        0.907
Kurtosis:                       3.478   Cond. No.                     1.63e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.63e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq112 ：gdp2_bindar_tj 行为方程 NO 系数表达不一样
gdp2_bindar_tj  = @coef(1)  + @coef(2) * investr_tj  + @coef(3) * gdp2_bindar_tj(-1)  + @coef(4) * d_gdp2_bindar_tj  + [ar(1) = @coef(5) , estsmpl = "1998 2017"]


In [39]:
test1 = df_ev[["gdp2_bindar_tj","investr_tj","d_gdp2_bindar_tj"]]
test1["gdp2_bindar_tj_s1"] = test1["gdp2_bindar_tj"].shift(1)
test1 = test1[(test1.index>1997) & (test1.index <2018)]
ar = 1 
ma = 0
mod = SARIMAX(endog = test1["gdp2_bindar_tj"] , exog = test1[["investr_tj","gdp2_bindar_tj_s1","d_gdp2_bindar_tj"]],order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()
results.summary() #results.summary().tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:         gdp2_bindar_tj   No. Observations:                   20
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 -68.338
Date:                Fri, 10 Jul 2020   AIC                            148.676
Time:                        08:24:05   BIC                            154.343
Sample:                             0   HQIC                           149.635
                                 - 20                                         
Covariance Type:                  opg                                         
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
intercept             2.7486      7.084      0.388      0.698     -11.136      16.633
investr_tj            0.0171      0.012      1.399      0.162      -0.007       0.041
gdp2_bindar_tj_s1     0.0693      0.215      0.321      0.748      -0.353       0.492
d_gdp2_bindar_tj     23.9886     23.055      1.040      0.298     -21.198      69.175
ar.L1                 1.0656      0.043     24.610      0.000       0.981       1.150
sigma2               75.3507     30.561      2.466      0.014      15.452     135.250
===================================================================================
Ljung-Box (Q):                       17.06   Jarque-Bera (JB):                 0.17
Prob(Q):                              0.52   Prob(JB):                         0.92
Heteroskedasticity (H):               0.36   Skew:                             0.11
Prob(H) (two-sided):                  0.24   Kurtosis:                         2.59
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

In [41]:
predict = results.predict(exog = test1[["investr_tj","gdp2_bindar_tj_s1","d_gdp2_bindar_tj"]] )
predict

1998     22.318889
1999    171.010299
2000    169.250078
2001    172.116547
2002    197.836235
2003    223.348145
2004    248.053692
2005    252.572503
2006    289.130691
2007    340.350306
2008    377.926519
2009    429.305271
2010    476.984877
2011    518.729771
2012    572.902581
2013    617.294722
2014    688.792221
2015    747.187798
2016    795.500637
2017    709.237585
dtype: float64

## Eq113 ：gdp3r_tj 行为方程  ok
行为方程 ：log(gdp3r_tj)  = @coef(1)  + @coef(2) * log(gdp2r_tj)  + @coef(3) * log(gdp3r_tj(-1))  + @coef(4) * log(exusd_tj * exrate_cn)  + @coef(5) * d_gdp3r_tj


In [42]:
test1 = df_ev[["gdp3r_tj","gdp2r_tj","exusd_tj","exrate_cn","d_gdp3r_tj"]]
test1["exusexrte"] = test1["exusd_tj"]*test1["exrate_cn"]
test1["gdp3r_tj_s1"] = test1["gdp3r_tj"].shift(1)           
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,gdp3r_tj,gdp2r_tj,exusd_tj,exrate_cn,d_gdp3r_tj,exusexrte,gdp3r_tj_s1
1997,1000.071558,627.011370,50.1818,828.98,0.0,41599.708564,885.109529
1998,1116.505019,665.212326,54.9856,827.91,0.0,45523.128096,1000.071558


In [44]:
results = sm.ols(formula = "np.log(gdp3r_tj) ~ np.log(gdp2r_tj)  + np.log(exusexrte) +  np.log(gdp3r_tj_s1) + d_gdp3r_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(gdp3r_tj)   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 7.793e+04
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           5.49e-36
Time:                        08:30:03   Log-Likelihood:                 82.482
No. Observations:                  22   AIC:                            -155.0
Df Residuals:                      17   BIC:                            -149.5
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               0.4584      0.065      7.001      0.000       0.320       0.596
np.log(gdp2r_tj)        0.2190      0.024      9.215      0.000       0.169       0.269
np.log(exusexrte)       0.0137      0.006      2.321      0.033       0.001       0.026
np.log(gdp3r_tj_s1)     0.7216      0.025     29.142      0.000       0.669       0.774
d_gdp3r_tj              0.0169      0.005      3.665      0.002       0.007       0.027
==============================================================================
Omnibus:                        9.027   Durbin-Watson:                   2.478
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                2.174
Skew:                          -0.219   Prob(JB):                        0.337
Kurtosis:                       1.523   Cond. No.                         872.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq114：con_fr_tj 定义方程 NO  结果不一致

定义方程 ： con_fr_tj  = con_pr_tj  + con_gr_tj


In [46]:
test1 = df_ev[["con_fr_tj" ,"con_pr_tj" , "con_gr_tj"]]
test1["验证结果"] = (test1["con_pr_tj"] + test1["con_gr_tj"])
test1.head(4)

,con_fr_tj,con_pr_tj,con_gr_tj,验证结果
1996,798.410306,647.575202,166.917470,814.492672
1997,881.427828,724.672159,176.263620,900.935780
1998,966.004365,780.385991,204.984272,985.370263
1999,1106.120795,830.410734,289.236065,1119.646799


## Eq115：con_gr_tj 定义方程 ok
定义方程 ： con_gr_tj  = con_gn_tj  / pcon_gd_tj  * 100

In [48]:
test1 = df_ev[["con_gr_tj" ,"con_gn_tj" , "pcon_gd_tj"]]
test1["预测"] = (test1["con_gn_tj"] /test1["pcon_gd_tj"])*100
test1.head(4)

,con_gr_tj,con_gn_tj,pcon_gd_tj,预测
1996,166.917470,123.97,74.270237,166.917470
1997,176.263620,131.88,74.819750,176.263620
1998,204.984272,152.64,74.464250,204.984272
1999,289.236065,212.98,73.635354,289.236065


## Eq116 ： con_pr_tj 定义方程 ok

定义方程 ： con_pr_tj  = con_pur_tj  + con_prr_tj


In [49]:
test1 = df_ev[["con_pr_tj" ,"con_pur_tj" , "con_prr_tj"]]
test1["预测"] = (test1["con_pur_tj"] + test1["con_prr_tj"])
test1.tail(4)

,con_pr_tj,con_pur_tj,con_prr_tj,预测
2017,5936.092487,5322.025218,614.067268,5936.092487
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq117 ：con_fn_tj 定义方程 ok
con_fn_tj  = con_pn_tj  + con_gn_tj

In [50]:
test1 = df_ev[["con_fn_tj" ,"con_pn_tj" , "con_gn_tj"]]
test1["预测"] = (test1["con_pn_tj"] + test1["con_gn_tj"])
test1.tail(4)

,con_fn_tj,con_pn_tj,con_gn_tj,预测
2017,8424.17,6078.15,2346.02,8424.17
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq118 ：con_pn_tj 定义方程 ok
con_pn_tj  = con_pun_tj  + con_prn_tj

In [51]:
test1 = df_ev[["con_pn_tj" ,"con_pun_tj" , "con_prn_tj"]]
test1["预测"] = (test1["con_pun_tj"] + test1["con_prn_tj"])
test1.tail(4)

,con_pn_tj,con_pun_tj,con_prn_tj,预测
2017,6078.15,5440.53,637.62,6078.15
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq119 ：con_pun_tj 定义方程 ok
con_pun_tj  = con_pur_tj  * pcon_pud_tj  / 100

In [52]:
test1 = df_ev[["con_pun_tj" ,"con_pur_tj","pcon_pud_tj"]]
test1["预测"] = (test1["con_pur_tj"] * test1["pcon_pud_tj"])/100
test1.tail(4)

,con_pun_tj,con_pur_tj,pcon_pud_tj,预测
2017,5440.53,5322.025218,102.226686,5440.53
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq120 ：con_prn_tj 定义方程 ok
con_prn_tj  = con_prr_tj  * pcon_prd_tj  / 100

In [53]:
test1 = df_ev[["con_prn_tj" ,"con_prr_tj" , "pcon_prd_tj"]]
test1["验证结果"] = (test1["con_prr_tj"] * test1["pcon_prd_tj"])/100
test1.tail(4)

,con_prn_tj,con_prr_tj,pcon_prd_tj,验证结果
2017,637.62,614.067268,103.83553,637.62
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq121 : con_gn_tj 行为方程 ok
con_gn_tj  = @coef(1)  + @coef(2) * d(fen_tj)  + @coef(3) * con_gn_tj(-1)

In [54]:
test1 = df_ev[["con_gn_tj","fen_tj"]]
test1["fen_tj_diff"] = test1["fen_tj"] - test1["fen_tj"].shift(1)
test1["con_gn_tj_s1"] = test1["con_gn_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1.head(2)

,con_gn_tj,fen_tj,fen_tj_diff,con_gn_tj_s1
1997,131.88,122.78,122.78,123.97
1998,152.64,137.93,15.15,131.88


In [56]:
results = sm.ols(formula = "con_gn_tj ~ fen_tj_diff + con_gn_tj_s1",data = test1).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              con_gn_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     3695.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           2.95e-24
Time:                        08:40:32   Log-Likelihood:                -106.03
No. Observations:                  21   AIC:                             218.1
Df Residuals:                      18   BIC:                             221.2
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       39.0602     14.708      2.656      0.016       8.161      69.960
fen_tj_diff      0.3061      0.048      6.317      0.000       0.204       0.408
con_gn_tj_s1     1.0207      0.013     78.552      0.000       0.993       1.048
==============================================================================
Omnibus:                        2.921   Durbin-Watson:                   1.052
Prob(Omnibus):                  0.232   Jarque-Bera (JB):                1.537
Skew:                           0.641   Prob(JB):                        0.464
Kurtosis:                       3.333   Cond. No.                     1.95e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.95e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq122 ： investn_tj 定义方程 ok
investn_tj  = investr_tj  * faipi_tj  / 100

In [57]:
test1 = df_ev[["investn_tj" ,"investr_tj" , "faipi_tj"]]
test1["预测"] = (test1["investr_tj"] * test1["faipi_tj"]) /100
test1.tail(4)

,investn_tj,investr_tj,faipi_tj,预测
2017,11274.6948,10875.121101,103.674200,11274.6948
2018,NaN,NaN,108.339539,NaN
2019,NaN,NaN,110.181311,NaN
2020,NaN,NaN,NaN,NaN


## Eq123 : inv_fon_tj 定义方程  ok
inv_fon_tj  = inv_for_tj  * pinv_fod_tj  / 100

In [58]:
test1 = df_ev[["inv_fon_tj" ,"inv_for_tj" , "pinv_fod_tj"]]
test1["预测"] = (test1["inv_for_tj"] * test1["pinv_fod_tj"])/100
test1.tail(4)

,inv_fon_tj,inv_for_tj,pinv_fod_tj,预测
2017,10467.18,10725.095104,97.595218,10467.18
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq124 : inv_chn_tj 定义方程 ok
inv_chn_tj  = inv_chr_tj  * pinv_chd_tj  / 100

In [59]:
test1 = df_ev[["inv_chn_tj" ,"inv_chr_tj" , "pinv_chd_tj"]]
test1["预测"] = (test1["inv_chr_tj"] * test1["pinv_chd_tj"])/100
test1.head(2)

,inv_chn_tj,inv_chr_tj,pinv_chd_tj,预测
1996,102.34,94.222611,108.615118,102.34
1997,105.84,96.188492,110.033953,105.84


## Eq 125 : consr_tj   定义方程  no 预测与真实值相差较大
consr_tj  = consn_tj  / rpi_tj  * 100

In [60]:
test1 = df_ev[["consr_tj" ,"consn_tj" , "rpi_tj"]]
test1["预测"] = (test1["consn_tj"] / test1["rpi_tj"])*100
test1.head(2)

,consr_tj,consn_tj,rpi_tj,预测
1996,NaN,NaN,91.846182,NaN
1997,NaN,NaN,92.489105,NaN


## Eq 126 :  inv_for_tj  定义方程 ok
inv_for_tj  = inv_fir_tj  + inv_chr_tj

In [61]:
test1 = df_ev[["inv_for_tj" ,"inv_fir_tj" , "inv_chr_tj"]]
test1["预测"] = test1["inv_fir_tj"] + test1["inv_chr_tj"]
test1.tail(4)

,inv_for_tj,inv_fir_tj,inv_chr_tj,预测
2017,10725.095104,10314.097004,410.9981,10725.095104
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq 127 : inv_fin_tj   定义方程 ok
inv_fin_tj  = inv_fir_tj  * pinv_fid_tj  / 100

In [62]:
test1 = df_ev[["inv_fin_tj" ,"inv_fir_tj" , "pinv_fid_tj"]]
test1["预测"] = (test1["inv_fir_tj"] * test1["pinv_fid_tj"])/100
test1.tail(4)

,inv_fin_tj,inv_fir_tj,pinv_fid_tj,预测
2017,10137.46,10314.097004,98.287422,10137.46
2018,NaN,NaN,NaN,NaN
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq 128 :  gdper_tj  定义方程 ok
gdper_tj  = con_fr_tj  + inv_for_tj  + nex_gsr_tj

In [63]:
test1 = df_ev[["gdper_tj" ,"con_fr_tj","inv_for_tj","nex_gsr_tj"]]
test1["预测"] = test1["con_fr_tj"] + test1["inv_for_tj"] + test1["nex_gsr_tj"]
test1.tail(4)

,gdper_tj,con_fr_tj,inv_for_tj,nex_gsr_tj,预测
2017,18982.612429,8208.670038,10725.095104,48.847287,18982.612429
2018,19665.986476,NaN,NaN,NaN,NaN
2019,20609.953827,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN


## Eq 129 : gdpen_tj   定义方程 ok
gdpen_tj  = con_fn_tj  + inv_fon_tj  + nex_gsn_tj

In [74]:
test1 = df_ev[["gdpen_tj" ,"con_fn_tj" , "inv_fon_tj","nex_gsn_tj"]]
test1["预测"] = test1["con_fn_tj"] + test1["inv_fon_tj"] + test1["nex_gsn_tj"]
test1.tail(4)

,gdpen_bj,con_fn_bj,inv_fon_bj,nex_gsn_bj,预测
2017,28014.9,16842.1,10946.30,226.5,28014.90
2018,30320.0,18508.9,11286.03,525.1,30320.03
2019,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN


## Eq 130 :  nex_gsn_tj   定义方程 ok
nex_gsn_tj  = nex_gsr_tj  * pnex_gsd_tj  / 100


In [65]:
test1 = df_ev[["nex_gsn_tj" ,"nex_gsr_tj" , "pnex_gsd_tj"]]
test1["预测"] = (test1["nex_gsr_tj"] * test1["pnex_gsd_tj"])/100
test1.tail(4)

,nex_gsn_tj,nex_gsr_tj,pnex_gsd_tj,预测
2017,-342.16,48.847287,-700.468794,-342.16
2018,NaN,NaN,683.630614,NaN
2019,NaN,NaN,-667.197197,NaN
2020,NaN,NaN,NaN,NaN


## Eq131 : con_pur_tj 行为方程 ok
con_pur_tj  = @coef(1)  + @coef(2) * yht_un_tj / pcon_pud_tj * 100  + @coef(3) * con_pur_tj(-1)  + [ar(1) = @coef(4) , estsmpl = "2006 2017"]

In [67]:
test1 = df_ev[["con_pur_tj" ,"yht_un_tj" , "pcon_pud_tj"]]
test1["yht_pcon_100"] = test1["yht_un_tj"]/test1["pcon_pud_tj"]*100
test1["con_pur_tj_s1"] = test1["con_pur_tj"].shift(1)
test1 = test1[(test1.index>2005) & (test1.index <2018)]
test1.head(2)

,con_pur_tj,yht_un_tj,pcon_pud_tj,yht_pcon_100,con_pur_tj_s1
2006,1394.296824,1080.002084,78.743118,1371.551071,1225.311880
2007,1595.094723,1281.642302,81.143871,1579.469025,1394.296824


In [68]:
ar = 1 
ma = 0
mod = SARIMAX(endog = test1["con_pur_tj"] , exog = test1[["yht_pcon_100","con_pur_tj_s1"]],order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()

results.summary() #results.summary().tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:             con_pur_tj   No. Observations:                   12
Model:               SARIMAX(1, 0, 0)   Log Likelihood                 -56.875
Date:                Fri, 10 Jul 2020   AIC                            123.750
Time:                        08:55:09   BIC                            125.739
Sample:                             0   HQIC                           122.496
                                 - 12                                         
Covariance Type:                  opg                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
intercept        60.7738     68.338      0.889      0.374     -73.166     194.713
yht_pcon_100      0.4686      0.244      1.921      0.055      -0.010       0.947
con_pur_tj_s1     0.5555      0.282      1.969      0.049       0.003       1.108
ar.L1             0.7083      0.515      1.375      0.169      -0.301       1.718
sigma2         1816.1087   1159.111      1.567      0.117    -455.707    4087.924
===================================================================================
Ljung-Box (Q):                       10.39   Jarque-Bera (JB):                 0.63
Prob(Q):                              0.41   Prob(JB):                         0.73
Heteroskedasticity (H):               0.24   Skew:                             0.49
Prob(H) (two-sided):                  0.20   Kurtosis:                         2.36
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Eq132 : con_prr_tj 行为方程 ok
eviews :function :d(x) :first difference ,d(x) = x - x(-1)

方程 ： con_prr_tj  = @coef(1)  + @coef(2) * yht_rn_tj / pcon_prd_tj * 100  + @coef(3) * con_prr_tj(-1)  + @coef(4) * d_con_prr_tj

In [69]:
test1 = df_ev[["con_prr_tj" ,"yht_rn_tj" , "pcon_prd_tj","d_con_prr_tj"]]
test1["yht_pcon_100"] = test1["yht_rn_tj"]/test1["pcon_prd_tj"]*100
test1["con_prr_tj_s1"] = test1["con_prr_tj"].shift(1)
test1 = test1[(test1.index>2007) & (test1.index <2019)]
test1.head(2)

,con_prr_bj,yht_rn_bj,pcon_prd_bj,d_con_prr_tj,yht_pcon_100,con_prr_tj_s1
2008,346.029075,268.777750,89.695232,0,299.656676,313.998622
2009,384.730111,305.178093,94.787089,0,321.961669,346.029075


In [70]:
results = sm.ols(formula = "con_prr_tj ~ yht_pcon_100 + con_prr_tj_s1 + d_con_prr_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             con_prr_bj   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.979
Method:                 Least Squares   F-statistic:                     157.9
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           8.63e-07
Time:                        08:57:29   Log-Likelihood:                -47.665
No. Observations:                  11   AIC:                             103.3
Df Residuals:                       7   BIC:                             104.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        29.0714     29.992      0.969      0.365     -41.848      99.991
yht_pcon_100      0.3971      0.285      1.394      0.206      -0.277       1.071
con_prr_tj_s1     0.6406      0.303      2.114      0.072      -0.076       1.357
d_con_prr_tj     63.7718     25.346      2.516      0.040       3.837     123.707
==============================================================================
Omnibus:                        2.736   Durbin-Watson:                   2.423
Prob(Omnibus):                  0.255   Jarque-Bera (JB):                0.411
Skew:                          -0.073   Prob(JB):                        0.814
Kurtosis:                       3.936   Cond. No.                     3.39e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.39e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq133 : investr_tj 行为方程
investr_tj  = @coef(1)  + @coef(2) * investr_tj(-1)  + @coef(3) * d(loann_tj) / faipi_tj * 100  + @coef(4) * d_investr_tj


In [74]:
test1 = df_ev[["investr_tj" ,"loann_tj" , "faipi_tj","d_investr_tj"]]
test1["investr_tj_s1"] = test1["investr_tj"].shift(1)
test1["loann_tj_diff"] = test1["loann_tj"] - test1["loann_tj"].shift(1) 
test1["loan_fai_100"] = test1["loann_tj_diff"]/test1["faipi_tj"]*100
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,investr_tj,loann_tj,faipi_tj,d_investr_tj,investr_tj_s1,loann_tj_diff,loan_fai_100
2006,2155.859512,5182.76,84.411808,0,1774.589218,NaN,NaN
2007,2720.257234,6241.07,86.606515,0,2155.859512,1058.31,1221.975039


In [75]:
results = sm.ols(formula = "investr_tj ~ investr_tj_s1 + loan_fai_100 + d_investr_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             investr_tj   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     574.8
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           9.77e-09
Time:                        09:06:40   Log-Likelihood:                -74.313
No. Observations:                  11   AIC:                             156.6
Df Residuals:                       7   BIC:                             158.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        80.3282    307.425      0.261      0.801    -646.616     807.272
investr_tj_s1     1.0124      0.029     34.875      0.000       0.944       1.081
loan_fai_100      0.4690      0.134      3.501      0.010       0.152       0.786
d_investr_tj   1715.4235    160.237     10.706      0.000    1336.523    2094.324
==============================================================================
Omnibus:                        8.990   Durbin-Watson:                   2.516
Prob(Omnibus):                  0.011   Jarque-Bera (JB):                4.177
Skew:                          -1.372   Prob(JB):                        0.124
Kurtosis:                       4.260   Cond. No.                     3.45e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.45e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq134 : inv_fir_tj 行为方程
inv_fir_tj  = @coef(1)  + @coef(2) * investr_tj  + @coef(3) * inv_fir_tj(-1)

In [81]:
test1 = df_ev[["inv_fir_tj" ,"investr_tj" ]]
test1["inv_fir_tj_s1"] = test1["inv_fir_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2018)]
test1.head(2)

,inv_fir_tj,investr_tj,inv_fir_tj_s1
1997,627.647701,630.655332,547.868876
1998,730.016960,736.162705,627.647701


In [82]:
results = sm.ols(formula = "inv_fir_tj ~ investr_tj + inv_fir_tj_s1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             inv_fir_tj   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.995
Method:                 Least Squares   F-statistic:                     1836.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           1.56e-21
Time:                        09:09:33   Log-Likelihood:                -144.88
No. Observations:                  21   AIC:                             295.8
Df Residuals:                      18   BIC:                             298.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       215.0497     86.032      2.500      0.022      34.304     395.795
investr_tj        0.2544      0.068      3.729      0.002       0.111       0.398
inv_fir_tj_s1     0.7273      0.091      7.966      0.000       0.535       0.919
==============================================================================
Omnibus:                        6.244   Durbin-Watson:                   0.788
Prob(Omnibus):                  0.044   Jarque-Bera (JB):                3.882
Skew:                           0.932   Prob(JB):                        0.144
Kurtosis:                       3.983   Cond. No.                     1.24e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.24e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq135 : nex_gsr_tj 行为方程
nex_gsr_tj  = @coef(1)  + @coef(2) * gdpr_tj  + @coef(3) * nex_gsr_tj(-1)  + @coef(4) * d_nex_gsr_tj

In [87]:
test1 = df_ev[["nex_gsr_tj","gdpr_tj" ,"d_nex_gsr_tj" ]]
test1["nex_gsr_tj_sum"] = test1["nex_gsr_tj"].shift(1) + test1["nex_gsr_tj"].shift(2)
test1 = test1[(test1.index>1997) & (test1.index <2018)]
test1.head(2)

,nex_gsr_tj,gdpr_tj,d_nex_gsr_tj,nex_gsr_tj_sum
1998,94.547979,1865.342166,0.0,183.084742
1999,139.352266,2051.845038,0.0,195.706870


In [88]:
results = sm.ols(formula = "nex_gsr_tj ~ gdpr_tj + nex_gsr_tj_sum + d_nex_gsr_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             nex_gsr_tj   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.982
Method:                 Least Squares   F-statistic:                     351.6
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           8.25e-15
Time:                        09:13:01   Log-Likelihood:                -111.12
No. Observations:                  20   AIC:                             230.2
Df Residuals:                      16   BIC:                             234.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept          8.0917     32.592      0.248      0.807     -61.001      77.184
gdpr_tj            0.0136      0.004      3.396      0.004       0.005       0.022
nex_gsr_tj_sum     0.4746      0.023     20.687      0.000       0.426       0.523
d_nex_gsr_tj     577.3317     28.592     20.192      0.000     516.719     637.944
==============================================================================
Omnibus:                        0.320   Durbin-Watson:                   2.707
Prob(Omnibus):                  0.852   Jarque-Bera (JB):                0.480
Skew:                          -0.090   Prob(JB):                        0.787
Kurtosis:                       2.263   Cond. No.                     2.16e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.16e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq136 : consn_tj 行为方程
- consn_tj  = @coef(1)  + @coef(2) * (yht_n_tj - yht_n_tj(-1))  + @coef(3) * consn_tj(-1)  + @coef(4) * d_consn_tj

- consn_tj  = @coef(1)  + @coef(2) * d(yht_n_tj)  + @coef(3) * consn_tj(-1)  + @coef(4) * d_consn_tj

In [89]:
test1 = df_ev[["consn_tj" ,"yht_n_tj" , "d_consn_tj"]]
test1["yht_n_tj_s1"] = test1["yht_n_tj"].shift(1)
test1["consn_tj_s1"] = test1["consn_tj"].shift(1)
test1["n_s1_diff"] = test1["yht_n_tj"] - test1["yht_n_tj_s1"]
test1.head(2)

,consn_tj,yht_n_tj,d_consn_tj,yht_n_tj_s1,consn_tj_s1,n_s1_diff
1996,NaN,NaN,0.0,NaN,NaN,NaN
1997,NaN,NaN,0.0,NaN,NaN,NaN


In [90]:
results = sm.ols(formula = "consn_tj ~ n_s1_diff +consn_tj_s1 + d_consn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               consn_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     1416.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           2.37e-12
Time:                        09:15:18   Log-Likelihood:                -73.681
No. Observations:                  13   AIC:                             155.4
Df Residuals:                       9   BIC:                             157.6
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      34.1068     98.338      0.347      0.737    -188.348     256.562
n_s1_diff       1.5906      0.393      4.047      0.003       0.701       2.480
consn_tj_s1     0.9143      0.029     32.062      0.000       0.850       0.979
d_consn_tj    158.8848     41.970      3.786      0.004      63.943     253.827
==============================================================================
Omnibus:                        1.037   Durbin-Watson:                   2.102
Prob(Omnibus):                  0.595   Jarque-Bera (JB):                0.722
Skew:                          -0.527   Prob(JB):                        0.697
Kurtosis:                       2.530   Cond. No.                     1.61e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq137: exusd_tj 行为方程
exusd_tj  = @coef(1)  + @coef(2) * exusd_ch  + @coef(3) * exusd_tj(-1)  + @coef(4) * d_exusd_tj  + [ar(1) = @coef(5) , ar(2) = @coef(6) , estsmpl = "1999 2018"]


In [91]:
test1 = df_ev[["exusd_tj" ,"exusd_ch" , "d_exusd_tj"]]
test1["exusd_tj_s1"] = test1["exusd_tj"].shift(1) 
#test1["exusd_tj_diff"] = test1["exusd_tj"] - test1["exusd_tj"].shift(1)
test1 = test1[(test1.index>1998) & (test1.index <2019)]
test1.head(2)

,exusd_tj,exusd_ch,d_exusd_tj,exusd_tj_s1,exusd_tj_diff
1999,63.3199,1949.31,0.0,54.9856,8.3343
2000,86.2906,2492.03,0.0,63.3199,22.9707


In [94]:
ar = (1 ,1)
ma = 0
mod = SARIMAX(endog = test1["exusd_tj"] , exog = test1[["exusd_ch","exusd_tj_s1" ,"d_exusd_tj"]],order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:               exusd_tj   No. Observations:                   20
Model:               SARIMAX(2, 0, 0)   Log Likelihood                 190.692
Date:                Fri, 10 Jul 2020   AIC                           -365.384
Time:                        09:21:32   BIC                           -358.261
Sample:                             0   HQIC                          -364.401
                                 - 20                                         
Covariance Type:                  opg                                         
=================================================================================
                    coef    std err          z      P>|z|      [0.025      0.975]
---------------------------------------------------------------------------------
intercept     -3.412e-14   1.46e-23  -2.33e+09      0.000   -3.41e-14   -3.41e-14
exusd_ch       1.214e-17   2.35e-19     51.637      0.000    1.17e-17    1.26e-17
exusd_tj_s1       1.0000   6.98e-22   1.43e+21      0.000       1.000       1.000
exusd_tj_diff     1.0000   5.63e-21   1.78e+20      0.000       1.000       1.000
d_exusd_tj     1.421e-14   2.68e-23   5.31e+08      0.000    1.42e-14    1.42e-14
ar.L1             1.0778   5.99e-36    1.8e+35      0.000       1.078       1.078
ar.L2            -0.1519   1.84e-36  -8.27e+34      0.000      -0.152      -0.152
sigma2             1e-10   4.67e-10      0.214      0.830   -8.16e-10    1.02e-09
===================================================================================
Ljung-Box (Q):                       14.60   Jarque-Bera (JB):                 0.88
Prob(Q):                              0.62   Prob(JB):                         0.65
Heteroskedasticity (H):               1.45   Skew:                            -0.10
Prob(H) (two-sided):                  0.66   Kurtosis:                         1.94
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
[2] Covariance matrix is singular or near-singular, with condition number    inf. Standard errors may be unstable.
"""

## Eq138 : imusd_tj 行为方程 ok  结果有细微差异
log(imusd_tj)  = @coef(1)  + @coef(2) * log(imusd_tj(-1))  + @coef(3) * log(exusd_tj)  + @coef(4) * d_imusd_tj

In [98]:
test1 = df_ev[["imusd_tj" ,"exusd_tj" , "d_imusd_tj"]]
test1["imusd_tj_s1"] = test1["imusd_tj"].shift(1)
test1.head(2)

,imusd_tj,exusd_tj,d_imusd_tj,imusd_tj_s1
1996,353.212159,40.4913,0.0,NaN
1997,414.877963,50.1818,0.0,353.212159


In [99]:
results = sm.ols(formula = "np.log(imusd_tj) ~ np.log(imusd_tj_s1) + np.log(exusd_tj) + d_imusd_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(imusd_tj)   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     1754.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           2.18e-23
Time:                        09:24:35   Log-Likelihood:                 36.491
No. Observations:                  23   AIC:                            -64.98
Df Residuals:                      19   BIC:                            -60.44
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=======================================================================================
                          coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------
Intercept               1.2849      0.127     10.147      0.000       1.020       1.550
np.log(imusd_tj_s1)     0.3504      0.069      5.103      0.000       0.207       0.494
np.log(exusd_tj)        0.6791      0.078      8.740      0.000       0.516       0.842
d_imusd_tj              0.1201      0.016      7.678      0.000       0.087       0.153
==============================================================================
Omnibus:                        1.372   Durbin-Watson:                   2.179
Prob(Omnibus):                  0.504   Jarque-Bera (JB):                1.126
Skew:                          -0.508   Prob(JB):                        0.569
Kurtosis:                       2.624   Cond. No.                         126.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq139  yht_n_tj  定义方程
yht_n_tj  = yht_rn_tj  + yht_un_tj

In [105]:
test1 = df_ev[["yht_n_tj" ,"yht_rn_tj","yht_un_tj"]]
test1["预测"] = test1["yht_rn_tj"] + test1["yht_un_tj"]
test1.tail(4)

,yht_n_bj,yht_rn_bj,yht_un_bj,预测
2017,12328.3000,711.2038,11617.0900,12328.2938
2018,13439.5858,770.3292,12669.2566,13439.5858
2019,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN


## Eq140 :yh_un_tj  定义方程
yh_un_tj  = yht_un_tj  / popu_tj

In [100]:
test1 = df_ev[["yh_un_tj" ,"yht_un_tj","popu_tj"]]
test1["预测"] = (test1["yht_un_tj"] / test1["popu_tj"])
test1.tail(4)

,yh_un_tj,yht_un_tj,popu_tj,预测
2017,40277.539313,5186.355000,0.129111,40169.737668
2018,42976.252637,5573.203418,0.129681,42976.252637
2019,46119.000000,NaN,0.130382,NaN
2020,NaN,NaN,NaN,NaN


## Eq141 : yh_rn_tj 定义方程
yh_rn_tj  = yht_rn_tj  / popr_tj


In [101]:
test1 = df_ev[["yh_rn_tj" ,"yht_rn_tj","popr_tj"]]
test1["预测"] = (test1["yht_rn_tj"] / test1["popr_tj"])
test1.tail(4)

,yh_rn_tj,yht_rn_tj,popr_tj,预测
2017,21753.675338,572.077600,0.026576,21526.098736
2018,23065.225282,606.131055,0.026279,23065.225282
2019,24804.000000,NaN,0.025801,NaN
2020,NaN,NaN,NaN,NaN


## Eq142 :  yhn_tj 定义方程
yhn_tj  = yht_n_tj  / totpop_tj

In [102]:
test1 = df_ev[["yhn_tj" ,"yht_n_tj","totpop_tj"]]
test1["预测"] = (test1["yht_n_tj"] / test1["totpop_tj"])
test1.tail(4)

,yhn_tj,yht_n_tj,totpop_tj,预测
2017,37022.0,5758.432000,0.155687,36987.237213
2018,39506.0,6179.334473,0.155960,39621.277721
2019,42404.0,NaN,0.156183,NaN
2020,NaN,NaN,NaN,NaN


## Eq143 :  conet_un_tj 定义方程
conet_un_tj  = cone_un_tj  * popu_tj

In [103]:
test1 = df_ev[["conet_un_tj" ,"cone_un_tj","popu_tj"]]
test1["预测"] = test1["cone_un_tj"] * test1["popu_tj"]
test1.tail(4)

,conet_un_tj,cone_un_tj,popu_tj,预测
2017,3944.828000,30283.646587,0.129111,3909.951895
2018,4234.747316,32655.109968,0.129681,4234.747316
2019,NaN,NaN,0.130382,NaN
2020,NaN,NaN,NaN,NaN


## Eq144   定义方程
conet_rn_tj  = cone_rn_tj  * popr_tj

In [104]:
test1 = df_ev[["conet_rn_tj" ,"cone_rn_tj","popr_tj"]]
test1["预测"] = test1["cone_rn_tj"] * test1["popr_tj"]
test1.tail(4)

,conet_rn_tj,cone_rn_tj,popr_tj,预测
2017,417.188300,16385.884207,0.026576,435.471259
2018,443.151411,16863.328539,0.026279,443.151411
2019,NaN,NaN,0.025801,NaN
2020,NaN,NaN,NaN,NaN


## Eq145 : yht_un_tj 行为方程  ok
yht_un_tj  = @coef(1)  + @coef(2) * (gdp2n_tj + gdp3n_tj - taxn_tj)  + @coef(3) * yht_un_tj(-1)


In [105]:
test1 = df_ev[["yht_un_tj","gdp2n_tj","gdp3n_tj","taxn_tj"]]
test1["yht_un_tj_s1"] = test1["yht_un_tj"].shift(1)
test1["gdp2gdp3tax"] = test1["gdp2n_tj"] + test1["gdp3n_tj"] - test1["taxn_tj"]
test1.head(2)

,yht_un_tj,gdp2n_tj,gdp3n_tj,taxn_tj,yht_un_tj_s1,gdp2gdp3tax
1996,NaN,609.10,445.16,NaN,NaN,NaN
1997,NaN,676.01,519.10,NaN,NaN,NaN


In [106]:
results = sm.ols(formula = "yht_un_tj ~ gdp2gdp3tax + yht_un_tj_s1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              yht_un_tj   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.169e+04
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           1.43e-17
Time:                        09:30:26   Log-Likelihood:                -62.779
No. Observations:                  13   AIC:                             131.6
Df Residuals:                      10   BIC:                             133.3
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       75.4956     32.575      2.318      0.043       2.914     148.077
gdp2gdp3tax      0.0659      0.011      6.223      0.000       0.042       0.089
yht_un_tj_s1     0.8389      0.034     24.670      0.000       0.763       0.915
==============================================================================
Omnibus:                        3.409   Durbin-Watson:                   2.473
Prob(Omnibus):                  0.182   Jarque-Bera (JB):                1.986
Skew:                           0.954   Prob(JB):                        0.370
Kurtosis:                       2.842   Cond. No.                     4.22e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.22e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq146 : yht_rn_bj 行为方程
yht_rn_tj  = @coef(1)  + @coef(2) * (gdp1n_tj + gdp3n_tj)  + @coef(3) * yht_rn_tj(-1)

In [108]:
test1 = df_ev[["yht_rn_tj","gdp1n_tj","gdp3n_tj"]]
test1["gdpn_tj_13sum"] = test1["gdp1n_tj"] + test1["gdp3n_tj"]
test1["yht_rn_tj_s1"] = test1["yht_rn_tj"].shift(1)
test1.head(2)

,yht_rn_tj,gdp1n_tj,gdp3n_tj,gdpn_tj_13sum,yht_rn_tj_s1
1996,NaN,67.67,445.16,512.83,NaN
1997,NaN,69.52,519.10,588.62,NaN


In [109]:
results = sm.ols(formula = "yht_rn_tj ~ gdpn_tj_13sum +  yht_rn_tj_s1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              yht_rn_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     2475.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           3.33e-14
Time:                        09:43:41   Log-Likelihood:                -43.205
No. Observations:                  13   AIC:                             92.41
Df Residuals:                      10   BIC:                             94.10
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept        44.4973      7.934      5.608      0.000      26.818      62.176
gdpn_tj_13sum     0.0277      0.007      3.729      0.004       0.011       0.044
yht_rn_tj_s1      0.4358      0.163      2.681      0.023       0.074       0.798
==============================================================================
Omnibus:                        0.537   Durbin-Watson:                   1.451
Prob(Omnibus):                  0.765   Jarque-Bera (JB):                0.583
Skew:                           0.270   Prob(JB):                        0.747
Kurtosis:                       2.115   Cond. No.                     2.68e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.68e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq147 : cone_un_tj 行为方程
cone_un_tj  = @coef(1)  + @coef(2) * yh_un_tj

In [111]:
test1 = df_ev[["cone_un_tj" , "yh_un_tj" ]]
test1.head(2)

,cone_un_tj,yh_un_tj
1996,4679.61,5967.0
1997,5204.15,6609.0


In [112]:
results = sm.ols(formula = "cone_un_tj ~ yh_un_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cone_un_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 2.642e+04
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           4.84e-34
Time:                        09:45:44   Log-Likelihood:                -159.29
No. Observations:                  23   AIC:                             322.6
Df Residuals:                      21   BIC:                             324.8
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    514.3755    103.148      4.987      0.000     299.867     728.884
yh_un_tj       0.7512      0.005    162.549      0.000       0.742       0.761
==============================================================================
Omnibus:                        0.598   Durbin-Watson:                   0.704
Prob(Omnibus):                  0.742   Jarque-Bera (JB):                0.642
Skew:                          -0.128   Prob(JB):                        0.726
Kurtosis:                       2.223   Cond. No.                     4.28e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.28e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq148 : cone_rn_tj 行为方程

cone_rn_tj  = @coef(1)  + @coef(2) * (yh_rn_tj - yh_rn_tj(-1))  + @coef(3) * cone_rn_tj(-1)  + @coef(4) * d_cone_rn_tj

In [114]:
test1 = df_ev[["cone_rn_tj" , "yh_rn_tj" ,"d_cone_rn_tj" ]]
test1["yh_rn_tj_diff"] = test1["yh_rn_tj"] - test1["yh_rn_tj"].shift(1)
test1["cone_rn_tj_s1"] = test1["cone_rn_tj"].shift(1)
test1.head(2)

,cone_rn_tj,yh_rn_tj,d_cone_rn_tj,yh_rn_tj_diff,cone_rn_tj_s1
1996,NaN,NaN,0,NaN,NaN
1997,NaN,NaN,0,NaN,NaN


In [115]:
results = sm.ols(formula = "cone_rn_tj ~ yh_rn_tj_diff + cone_rn_tj_s1 + d_cone_rn_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             cone_rn_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2096.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           1.06e-17
Time:                        09:48:33   Log-Likelihood:                -117.31
No. Observations:                  17   AIC:                             242.6
Df Residuals:                      13   BIC:                             246.0
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept      -143.2606    147.338     -0.972      0.349    -461.565     175.044
yh_rn_tj_diff     1.3571      0.165      8.217      0.000       1.000       1.714
cone_rn_tj_s1     0.9252      0.019     48.942      0.000       0.884       0.966
d_cone_rn_tj     78.0178     29.592      2.636      0.021      14.089     141.946
==============================================================================
Omnibus:                        0.852   Durbin-Watson:                   1.286
Prob(Omnibus):                  0.653   Jarque-Bera (JB):                0.601
Skew:                           0.429   Prob(JB):                        0.741
Kurtosis:                       2.664   Cond. No.                     2.03e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq149 : frn_tj 行为方程
frn_tj  = @coef(1)  + @coef(2) * taxn_tj  + @coef(3) * d(frn_tj)  + @coef(4) * d_frn_tj


In [121]:
test1 = df_ev[["frn_tj","taxn_tj","d_frn_tj"]]
test1["frn_tj_diff"] = test1["frn_tj"] - test1["frn_tj"].shift(1)
test1 = test1[(test1.index>2000) & (test1.index <2019)]
test1.head(2)

,frn_tj,taxn_tj,d_frn_tj,frn_tj_diff
2001,163.64,153.8020,0.0,30.03
2002,171.83,154.8131,0.0,8.19


In [122]:
results = sm.ols(formula = "frn_tj ~ taxn_tj + frn_tj_diff + d_frn_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 frn_tj   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     1283.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           2.58e-17
Time:                        09:54:25   Log-Likelihood:                -97.865
No. Observations:                  18   AIC:                             203.7
Df Residuals:                      14   BIC:                             207.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    -120.7810     28.553     -4.230      0.001    -182.021     -59.541
taxn_tj         1.5207      0.030     51.360      0.000       1.457       1.584
frn_tj_diff     0.6164      0.084      7.322      0.000       0.436       0.797
d_frn_tj      129.4358     18.928      6.838      0.000      88.839     170.033
==============================================================================
Omnibus:                        1.664   Durbin-Watson:                   1.784
Prob(Omnibus):                  0.435   Jarque-Bera (JB):                1.393
Skew:                          -0.575   Prob(JB):                        0.498
Kurtosis:                       2.268   Cond. No.                     1.98e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.98e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq150 ： taxn_tj1  no d_taxn_tj_1是模拟结果数据？
log(taxn_tj1)  = @coef(1)  + @coef(2) * log(tax_vadn_tj + tax_cinn_tj + tax_pinn_tj)  + @coef(3) * d_taxn_tj_1 

In [125]:
test1 = df_ev[["taxn_tj1","tax_vadn_tj","tax_cinn_tj","tax_pinn_tj","d_taxn_tj_1"]]
test1["tax_sum"] = test1["tax_vadn_tj"] + test1["tax_cinn_tj"] + test1["tax_pinn_tj"]
test1 = test1[(test1.index>1997) & (test1.index <2019)]
test1.head(2)

,taxn_tj1,tax_vadn_tj,tax_cinn_tj,tax_pinn_tj,d_taxn_tj_1,tax_sum
1998,NaN,18.99,20.34,7.91,0.0,47.24
1999,NaN,20.83,24.02,8.86,0.0,53.71


In [126]:
results = sm.ols(formula = "np.log(taxn_tj1) ~ np.log(tax_sum) + d_taxn_tj_1 ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:       np.log(taxn_tj1)   R-squared:                       0.994
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     1243.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           2.16e-17
Time:                        09:58:17   Log-Likelihood:                 23.131
No. Observations:                  18   AIC:                            -40.26
Df Residuals:                      15   BIC:                            -37.59
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept           0.4388      0.123      3.557      0.003       0.176       0.702
np.log(tax_sum)     1.0835      0.023     48.074      0.000       1.035       1.132
d_taxn_tj_1         0.2046      0.019     10.930      0.000       0.165       0.245
==============================================================================
Omnibus:                        2.610   Durbin-Watson:                   1.536
Prob(Omnibus):                  0.271   Jarque-Bera (JB):                1.151
Skew:                           0.122   Prob(JB):                        0.562
Kurtosis:                       1.785   Cond. No.                         42.1
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq151 taxn_tj2 行为方程  no d_taxn_tj_2非数据源的话，需要衍生规则

taxn_tj2  = @coef(1)  + @coef(2) * gdpn_tj  + @coef(3) * d_taxn_tj_2

In [132]:
test1 = df_ev[["taxn_tj2","gdpn_tj","d_taxn_tj_2"]]
test1.head(2)

,taxn_tj2,gdpn_tj,d_taxn_tj_2
1996,NaN,1121.93,0.0
1997,NaN,1264.63,0.0


In [133]:
results = sm.ols(formula = "taxn_tj2 ~ gdpn_tj+ d_taxn_tj_2 ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               taxn_tj2   R-squared:                       0.974
Model:                            OLS   Adj. R-squared:                  0.971
Method:                 Least Squares   F-statistic:                     304.5
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           1.84e-13
Time:                        10:02:12   Log-Likelihood:                -113.01
No. Observations:                  19   AIC:                             232.0
Df Residuals:                      16   BIC:                             234.9
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -69.4325     46.035     -1.508      0.151    -167.022      28.157
gdpn_tj         0.0947      0.005     20.372      0.000       0.085       0.105
d_taxn_tj_2    31.6849     79.294      0.400      0.695    -136.410     199.780
==============================================================================
Omnibus:                       39.765   Durbin-Watson:                   1.350
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              114.465
Skew:                           3.200   Prob(JB):                     1.39e-25
Kurtosis:                      13.180   Cond. No.                     4.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq152 : taxn_tj 定义方程式
taxn_tj  = (taxn_tj1  + taxn_tj2)  / 2

In [135]:
test1 = df_ev[["taxn_tj","taxn_tj1","taxn_tj2"]]
test1["预测"] = (test1["taxn_tj1"] + test1["taxn_tj2"])/2
test1.tail(2)

,taxn_tj,taxn_tj1,taxn_tj2,预测
2019,1634.2,1634.2,1634.2,1634.2
2020,NaN,NaN,NaN,NaN


## Eq153 : tax_vadn_tj 行为方程式  ok
log(tax_vadn_tj)  = @coef(1)  + @coef(2) * log(gdpn_tj)  + @coef(3) * log(tax_vadn_tj(-1))  + @coef(4) * d_tax_vadn_tj

In [140]:
test1 = df_ev[["tax_vadn_tj","gdpn_tj","d_tax_vadn_tj"]]
test1["tax_vadn_tj_s1"] = test1["tax_vadn_tj"].shift(1)
test1["log_tax_vadn"] = np.log(test1["tax_vadn_tj"])
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,tax_vadn_tj,gdpn_tj,d_tax_vadn_tj,tax_vadn_tj_s1,log_tax_vadn
1997,17.30,1264.63,0.0,17.25,2.850707
1998,18.99,1374.60,0.0,17.30,2.943913


In [141]:
results = sm.ols(formula = "np.log(tax_vadn_tj) ~ np.log(gdpn_tj) + np.log(tax_vadn_tj_s1) + d_tax_vadn_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                             OLS Regression Results                            
===============================================================================
Dep. Variable:     np.log(tax_vadn_tj)   R-squared:                       0.996
Model:                             OLS   Adj. R-squared:                  0.995
Method:                  Least Squares   F-statistic:                     1519.
Date:                 Fri, 10 Jul 2020   Prob (F-statistic):           7.94e-22
Time:                         10:06:04   Log-Likelihood:                 27.781
No. Observations:                   22   AIC:                            -47.56
Df Residuals:                       18   BIC:                            -43.20
Df Model:                            3                                         
Covariance Type:             nonrobust                                         
==========================================================================================
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -0.5962      0.330     -1.807      0.088      -1.289       0.097
np.log(gdpn_tj)            0.1542      0.066      2.324      0.032       0.015       0.294
np.log(tax_vadn_tj_s1)     0.8685      0.059     14.623      0.000       0.744       0.993
d_tax_vadn_tj              0.3501      0.033     10.480      0.000       0.280       0.420
==============================================================================
Omnibus:                        0.732   Durbin-Watson:                   1.680
Prob(Omnibus):                  0.694   Jarque-Bera (JB):                0.118
Skew:                          -0.161   Prob(JB):                        0.943
Kurtosis:                       3.158   Cond. No.                         207.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq154 : tax_cinn_tj 行为方程 ok
tax_cinn_tj  = @coef(1)  + @coef(2) * gdpn_tj  + @coef(3) * tax_cinn_tj(-1)

In [5]:
test1 = df_ev[["tax_cinn_tj","gdpn_tj"]]
test1["tax_cinn_tj_s1"] = test1["tax_cinn_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,tax_cinn_tj,gdpn_tj,tax_cinn_tj_s1
1997,21.48,1264.63,19.06
1998,20.34,1374.60,21.48


In [6]:
results = sm.ols(formula = "tax_cinn_tj ~ gdpn_tj+ tax_cinn_tj_s1 ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            tax_cinn_tj   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1056.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           3.35e-20
Time:                        03:58:37   Log-Likelihood:                -81.204
No. Observations:                  22   AIC:                             168.4
Df Residuals:                      19   BIC:                             171.7
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -3.4952      3.979     -0.878      0.391     -11.823       4.833
gdpn_tj            0.0089      0.002      4.655      0.000       0.005       0.013
tax_cinn_tj_s1     0.4892      0.126      3.869      0.001       0.225       0.754
==============================================================================
Omnibus:                        0.497   Durbin-Watson:                   2.041
Prob(Omnibus):                  0.780   Jarque-Bera (JB):                0.130
Skew:                           0.188   Prob(JB):                        0.937
Kurtosis:                       2.977   Cond. No.                     1.85e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq155 ： tax_pinn_tj 行为方程
tax_pinn_tj  = @coef(1)  + @coef(2) * yht_n_tj  + @coef(3) * tax_pinn_tj(-1) + @coef(4) * d_tax_pinn_tj

In [7]:
test1 = df_ev[["tax_pinn_tj","yht_n_tj","d_tax_pinn_tj"]]
test1["tax_pinn_tj_s1"] = test1["tax_pinn_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,tax_pinn_tj,yht_n_tj,d_tax_pinn_tj,tax_pinn_tj_s1
1997,6.49,NaN,0,5.30
1998,7.91,NaN,0,6.49


In [8]:
results = sm.ols(formula = "tax_pinn_tj ~ yht_n_tj+ tax_pinn_tj_s1 + d_tax_pinn_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:            tax_pinn_tj   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     715.5
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           5.79e-12
Time:                        04:00:27   Log-Likelihood:                -31.540
No. Observations:                  14   AIC:                             71.08
Df Residuals:                      10   BIC:                             73.64
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.1091      1.740     -0.063      0.951      -3.987       3.769
yht_n_tj           0.0035      0.002      1.865      0.092      -0.001       0.008
tax_pinn_tj_s1     0.9476      0.107      8.820      0.000       0.708       1.187
d_tax_pinn_tj      5.8034      1.414      4.104      0.002       2.652       8.954
==============================================================================
Omnibus:                        1.555   Durbin-Watson:                   2.634
Prob(Omnibus):                  0.460   Jarque-Bera (JB):                0.995
Skew:                           0.628   Prob(JB):                        0.608
Kurtosis:                       2.642   Cond. No.                     9.05e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.05e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq156 ： fen_bj1 行为方程
fen_tj1  = @coef(1)  + @coef(2) * frn_tj  + @coef(3) * d_fen_tj_1  + [ar(1) = @coef(4) , estsmpl = "1998 2018"]

In [15]:
test1 = df_ev[["fen_tj1","frn_tj","d_fen_tj_1"]]
test1.head(2)

,fen_tj1,frn_tj,d_fen_tj_1
1996,NaN,0.00,0.0
1997,122.78,89.91,0.0


In [16]:
test1 = test1[(test1.index>1997) & (test1.index <2019)]
ar = 1 
ma = 0
mod = SARIMAX(endog = test1["fen_tj1"] , exog = test1[["frn_tj","d_fen_tj_1"]] ,order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()
results.summary() #results.summary().tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:                fen_tj1   No. Observations:                   21
Model:               SARIMAX(1, 0, 0)   Log Likelihood                -113.628
Date:                Mon, 13 Jul 2020   AIC                            237.256
Time:                        04:03:27   BIC                            242.235
Sample:                             0   HQIC                           238.228
                                 - 21                                         
Covariance Type:                  opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept     32.5399     74.326      0.438      0.662    -113.136     178.216
frn_tj         0.5816      0.102      5.692      0.000       0.381       0.782
d_fen_tj_1    70.8188      7.990      8.863      0.000      55.158      86.479
ar.L1          1.0897      0.065     16.756      0.000       0.962       1.217
sigma2      5010.5906   2807.723      1.785      0.074    -492.446    1.05e+04
===================================================================================
Ljung-Box (Q):                        5.42   Jarque-Bera (JB):                 0.08
Prob(Q):                              1.00   Prob(JB):                         0.96
Heteroskedasticity (H):              21.24   Skew:                             0.16
Prob(H) (two-sided):                  0.00   Kurtosis:                         3.04
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Eq157 ： fen_tj_2 行为方程 ok 数据源d_fen_tj_2 ？
fen_tj2  = @coef(1)  + @coef(2) * (fe_edun_tj + fe_tecn_tj + fe_culn_tj + fe_ssen_tj + fe_medn_tj + fe_eepn_tj)  + @coef(3) * d_fen_tj_2

In [19]:
test1 = df_ev[["fen_tj2","fe_edun_tj","fe_tecn_tj","fe_culn_tj","fe_ssen_tj","fe_medn_tj","fe_eepn_tj","d_fen_tj_2"]]
test1["fen_sum"] = test1[["fe_edun_tj","fe_tecn_tj","fe_culn_tj","fe_ssen_tj","fe_medn_tj","fe_eepn_tj"]].sum(axis =1)
test1 = test1[(test1.index>2006) & (test1.index <2019)]
test1.head(2)

,fen_tj2,fe_edun_tj,fe_tecn_tj,fe_culn_tj,fe_ssen_tj,fe_medn_tj,fe_eepn_tj,d_fen_tj_2,fen_sum
2007,674.33,110.0248,22.3384,15.9616,79.5714,33.0964,58.0130,0,319.0056
2008,867.72,141.6985,28.6530,18.0120,105.7164,41.9156,10.9789,0,346.9744


In [20]:
results = sm.ols(formula = "fen_tj2 ~ fen_sum + d_fen_tj_2 ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                fen_tj2   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     1385.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           6.26e-12
Time:                        04:06:37   Log-Likelihood:                -65.587
No. Observations:                  12   AIC:                             137.2
Df Residuals:                       9   BIC:                             138.6
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     33.5474     46.566      0.720      0.490     -71.792     138.887
fen_sum        2.4715      0.048     51.354      0.000       2.363       2.580
d_fen_tj_2   142.1044     18.430      7.711      0.000     100.413     183.796
==============================================================================
Omnibus:                        6.807   Durbin-Watson:                   0.836
Prob(Omnibus):                  0.033   Jarque-Bera (JB):                2.948
Skew:                          -1.045   Prob(JB):                        0.229
Kurtosis:                       4.236   Cond. No.                     2.37e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.37e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq158 ： fen_tj 定义方程
fen_tj  = (fen_tj1  + fen_tj2)  / 2

In [21]:
test1 = df_ev[["fen_tj","fen_tj1","fen_tj2"]]
test1["预测"] = (test1["fen_tj1"]+test1["fen_tj2"])/2
test1.head(2)

,fen_tj,fen_tj1,fen_tj2,预测
1996,0.00,NaN,NaN,NaN
1997,122.78,122.78,122.78,122.78


## Eq159 :  depn_tj 行为方程  ok  
log(depn_tj)  = @coef(1)  + @coef(2) * log(dep_san_tj)  + @coef(3) * log(depn_tj(-1))  + @coef(4) * d_depn_tj

In [22]:
test1 = df_ev[["depn_tj","dep_san_tj","d_depn_tj"]]
test1["depn_tj_s1"] = test1["depn_tj"].shift(1)
test1 = test1[(test1.index>2004) & (test1.index <2019)]
test1.head(2)

,depn_tj,dep_san_tj,d_depn_tj,depn_tj_s1
2005,5716.09,2462.66,0.0,4750.28
2006,6564.47,2811.71,0.0,5716.09


In [23]:
results = sm.ols(formula = "np.log(depn_tj) ~ np.log(dep_san_tj) + np.log(depn_tj_s1) + d_depn_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        np.log(depn_tj)   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     1678.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           8.27e-14
Time:                        04:09:03   Log-Likelihood:                 31.840
No. Observations:                  14   AIC:                            -55.68
Df Residuals:                      10   BIC:                            -53.12
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
======================================================================================
                         coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------
Intercept             -0.3918      0.254     -1.545      0.153      -0.957       0.173
np.log(dep_san_tj)     0.8307      0.153      5.444      0.000       0.491       1.171
np.log(depn_tj_s1)     0.3019      0.117      2.590      0.027       0.042       0.562
d_depn_tj              0.1310      0.029      4.585      0.001       0.067       0.195
==============================================================================
Omnibus:                        0.890   Durbin-Watson:                   2.867
Prob(Omnibus):                  0.641   Jarque-Bera (JB):                0.757
Skew:                           0.300   Prob(JB):                        0.685
Kurtosis:                       2.031   Cond. No.                         503.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq160 : dep_san_tj  行为方程  ok  
dep_san_tj  = @coef(1)  + @coef(2) * d(yht_n_tj)  + @coef(3) * dep_san_tj(-1)  + @coef(4) * d_dep_san_tj

In [26]:
test1 = df_ev[["dep_san_tj","yht_n_tj","d_dep_san_tj"]]
test1["yht_n_tj_diff"] = test1["yht_n_tj"] - test1["yht_n_tj"].shift(1)
test1["dep_san_tj_s1"] = test1["dep_san_tj"].shift(1)
test1 = test1[(test1.index>2001) & (test1.index <2019)]
test1.head(2)

,dep_san_tj,yht_n_tj,d_dep_san_tj,yht_n_tj_diff,dep_san_tj_s1
2002,1486.38,NaN,0.0,NaN,1284.95
2003,1825.32,NaN,0.0,NaN,1486.38


In [27]:
results = sm.ols(formula = "dep_san_tj ~ yht_n_tj_diff + dep_san_tj_s1 + d_dep_san_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             dep_san_tj   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.993
Method:                 Least Squares   F-statistic:                     564.3
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.47e-10
Time:                        04:12:57   Log-Likelihood:                -85.782
No. Observations:                  13   AIC:                             179.6
Df Residuals:                       9   BIC:                             181.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       292.5599    240.722      1.215      0.255    -251.990     837.110
yht_n_tj_diff     2.1487      1.012      2.123      0.063      -0.141       4.438
dep_san_tj_s1     0.9038      0.047     19.093      0.000       0.797       1.011
d_dep_san_tj     95.2220     27.136      3.509      0.007      33.837     156.607
==============================================================================
Omnibus:                        0.447   Durbin-Watson:                   2.409
Prob(Omnibus):                  0.800   Jarque-Bera (JB):                0.533
Skew:                           0.240   Prob(JB):                        0.766
Kurtosis:                       2.132   Cond. No.                     2.66e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.66e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq161 :  loan_ln_tj 行为方程  ok

log(loan_ln_bj)  = @coef(1)  + @coef(2) * log(depn_bj)  

In [28]:
test1 = df_ev[["loan_ln_tj","depn_tj"]]
test1 = test1[(test1.index>2003) & (test1.index <2019)]
test1.head(2)

,loan_ln_tj,depn_tj
2004,1789.90,4750.28
2005,2204.62,5716.09


In [29]:
results = sm.ols(formula = "np.log(loan_ln_tj) ~ np.log(depn_tj)  ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:     np.log(loan_ln_tj)   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     1704.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           3.57e-15
Time:                        04:14:42   Log-Likelihood:                 19.489
No. Observations:                  15   AIC:                            -34.98
Df Residuals:                      13   BIC:                            -33.56
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          -2.7961      0.284     -9.843      0.000      -3.410      -2.182
np.log(depn_tj)     1.2181      0.030     41.283      0.000       1.154       1.282
==============================================================================
Omnibus:                        1.505   Durbin-Watson:                   0.752
Prob(Omnibus):                  0.471   Jarque-Bera (JB):                0.877
Skew:                           0.161   Prob(JB):                        0.645
Kurtosis:                       1.860   Cond. No.                         151.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq162 :  loann_tj 行为方程 ok

loann_tj  = @coef(1)  + @coef(2) * depn_tj  + @coef(3) * loann_tj(-1)  + @coef(4) * intratel_cn

In [30]:
test1 = df_ev[["loann_tj","depn_tj","intratel_cn"]]
test1["loann_tj_s1"] = test1["loann_tj"].shift(1)
test1 = test1[(test1.index>2006) & (test1.index <2019)]
test1.head(2)

,loann_tj,depn_tj,intratel_cn,loann_tj_s1
2007,6241.07,7930.31,6.717452,5182.76
2008,7383.29,9606.36,7.172705,6241.07


In [31]:
results = sm.ols(formula = "loann_tj ~ depn_tj+ loann_tj_s1  + intratel_cn  ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               loann_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     3587.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           7.49e-13
Time:                        04:24:27   Log-Likelihood:                -82.130
No. Observations:                  12   AIC:                             172.3
Df Residuals:                       8   BIC:                             174.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept    4994.5701   1210.996      4.124      0.003    2202.008    7787.132
depn_tj         0.1769      0.056      3.182      0.013       0.049       0.305
loann_tj_s1     0.8223      0.049     16.639      0.000       0.708       0.936
intratel_cn  -610.0670    150.466     -4.055      0.004    -957.042    -263.092
==============================================================================
Omnibus:                        1.134   Durbin-Watson:                   2.281
Prob(Omnibus):                  0.567   Jarque-Bera (JB):                0.801
Skew:                           0.308   Prob(JB):                        0.670
Kurtosis:                       1.894   Cond. No.                     4.33e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.33e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq 163 : emp1_tj 定义方程 ok

emp1_tj  = totemp_tj  - emp2_tj  - emp3_tj

In [35]:
test1 = df_ev[["emp1_tj" , "totemp_tj" , "emp2_tj" , "emp3_tj"]]
test1["预测"] = test1["totemp_tj"] - test1["emp2_tj"] - test1["emp3_tj"]
test1.tail(4)

,emp1_tj,totemp_tj,emp2_tj,emp3_tj,预测
2017,62.71,894.83,290.90,541.22,62.71
2018,60.07,896.56,285.02,551.47,60.07
2019,NaN,NaN,NaN,NaN,NaN
2020,NaN,NaN,NaN,NaN,NaN


## Eq164 : popr_tj 定义方程
popr_tj  = totpop_tj  - popu_tj

In [36]:
test1 = df_ev[["popr_tj" , "totpop_tj" , "popu_tj" ]]
test1["预测"] = test1["totpop_tj"] - test1["popu_tj"] 
test1.head(2)

,popr_tj,totpop_tj,popu_tj,预测
1996,NaN,0.094819,NaN,NaN
1997,NaN,0.095259,NaN,NaN


## Eq165 ： totemp_tj 行为方程 ok
totemp_tj  = @coef(1)  + @coef(2) * @pch(gdpr_tj)  + @coef(3) * totemp_tj(-1)

In [7]:
test1 = df_ev[["totemp_tj","gdpr_tj"]]
test1["gdpr_tj_pch"] =( test1["gdpr_tj"] - test1["gdpr_tj"].shift(1))/test1["gdpr_tj"].shift(1)
test1["totemp_tj_s1"] = test1["totemp_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,totemp_tj,gdpr_tj,gdpr_tj_pch,totemp_tj_s1
1997,513.33,1706.422913,0.120856,512.00
1998,508.10,1865.342166,0.093130,513.33


In [8]:
results = sm.ols(formula = "totemp_tj ~ gdpr_tj_pch + totemp_tj_s1  ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              totemp_tj   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     2775.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           3.66e-24
Time:                        06:37:31   Log-Likelihood:                -80.390
No. Observations:                  22   AIC:                             166.8
Df Residuals:                      19   BIC:                             170.1
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     -101.4715     16.090     -6.306      0.000    -135.149     -67.794
gdpr_tj_pch    503.8051     64.591      7.800      0.000     368.614     638.996
totemp_tj_s1     1.0869      0.016     68.152      0.000       1.053       1.120
==============================================================================
Omnibus:                        2.562   Durbin-Watson:                   2.068
Prob(Omnibus):                  0.278   Jarque-Bera (JB):                1.029
Skew:                           0.340   Prob(JB):                        0.598
Kurtosis:                       3.813   Cond. No.                     2.06e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.06e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq166 : emp2_tj 行为方程  ok
emp2_tj  = @coef(1)  + @coef(2) * @pch(gdp2r_tj)  + @coef(3) * emp2_tj(-1)  + @coef(4) * d_emp2_tj

In [14]:
test1 = df_ev[["emp2_tj","gdp2r_tj","d_emp2_tj"]]
test1["gdp2r_tj_pch"] =  (test1["gdp2r_tj"] - test1["gdp2r_tj"].shift(1) )/  test1["gdp2r_tj"].shift(1)  #增长率
test1["emp2_tj_s1"] = test1["emp2_tj"].shift(1)
test1 = test1[(test1.index>2006) & (test1.index <2019)]
test1.head(2)

,emp2_tj,gdp2r_tj,d_emp2_tj,gdp2r_tj_pch,emp2_tj_s1
2007,261.35,2512.516924,0.0,0.169069,234.85
2008,271.90,2981.661424,-0.5,0.186723,261.35


In [15]:
results = sm.ols(formula = "emp2_tj ~  gdp2r_tj_pch   + emp2_tj_s1 + d_emp2_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                emp2_tj   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     364.4
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           6.83e-09
Time:                        06:44:19   Log-Likelihood:                -27.148
No. Observations:                  12   AIC:                             62.30
Df Residuals:                       8   BIC:                             64.24
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept       64.8717      9.616      6.746      0.000      42.697      87.046
gdp2r_tj_pch   125.1391     14.022      8.924      0.000      92.804     157.475
emp2_tj_s1       0.7424      0.029     25.743      0.000       0.676       0.809
d_emp2_tj       14.1692      1.217     11.644      0.000      11.363      16.975
==============================================================================
Omnibus:                        5.251   Durbin-Watson:                   2.597
Prob(Omnibus):                  0.072   Jarque-Bera (JB):                1.310
Skew:                          -0.066   Prob(JB):                        0.520
Kurtosis:                       1.387   Cond. No.                     5.71e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 5.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq 167 : emp3_tj 行为方程 ok 
emp3_tj  = @coef(1)  + @coef(2) * @pch(gdp3r_tj)  + @coef(3) * emp3_tj(-1)  + @coef(4) * d_emp3_tj

In [19]:
test1 = df_ev[["emp3_tj","gdp3r_tj" ,"d_emp3_tj"]]
test1["gdp3r_tj_pch"] =  (test1["gdp3r_tj"] - test1["gdp3r_tj"].shift(1) )/  test1["gdp3r_tj"].shift(1)  #增长率
test1["emp3_tj_s1"] = test1["emp3_tj"].shift(1)
test1 = test1[(test1.index>2006) & (test1.index <2019)]
test1.head(2)

,emp3_tj,gdp3r_tj,d_emp3_tj,gdp3r_tj_pch,emp3_tj_s1
2007,275.60,3190.305410,0.0,0.152039,246.96
2008,299.12,3689.998196,0.0,0.156629,275.60


In [20]:
results = sm.ols(formula = "emp3_tj ~ gdp3r_tj_pch + emp3_tj_s1  + d_emp3_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                emp3_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     3424.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           9.02e-13
Time:                        06:46:40   Log-Likelihood:                -28.659
No. Observations:                  12   AIC:                             65.32
Df Residuals:                       8   BIC:                             67.26
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        1.7129     21.404      0.080      0.938     -47.645      51.071
gdp3r_tj_pch   162.7237     83.317      1.953      0.087     -29.406     354.853
emp3_tj_s1       1.0004      0.030     33.904      0.000       0.932       1.068
d_emp3_tj       24.8958      2.537      9.811      0.000      19.044      30.747
==============================================================================
Omnibus:                        0.103   Durbin-Watson:                   3.003
Prob(Omnibus):                  0.950   Jarque-Bera (JB):                0.291
Skew:                          -0.153   Prob(JB):                        0.864
Kurtosis:                       2.301   Cond. No.                     3.75e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq168 ： totpop_tj 行为方程  ok
totpop_tj  = @coef(1)  + @coef(2) * d(gdpr_tj)  + @coef(3) * totpop_tj(-1)  + @coef(4) * d_totpop_tj


In [21]:
test1 = df_ev[["totpop_tj","gdpr_tj","d_totpop_tj"]]
test1["gdpr_tj_diff"] = test1["gdpr_tj"] - test1["gdpr_tj"].shift(1)
test1["totpop_tj_s1"] = test1["totpop_tj"].shift(1)
test1 = test1[(test1.index>1997) & (test1.index <2019)]
test1.head(2)

,totpop_tj,gdpr_tj,d_totpop_tj,gdpr_tj_diff,totpop_tj_s1
1998,0.095664,1865.342166,0,158.919254,0.095259
1999,0.095948,2051.845038,0,186.502872,0.095664


In [22]:
results = sm.ols(formula = "totpop_tj ~  totpop_tj_s1  + gdpr_tj_diff + d_totpop_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              totpop_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.003e+04
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           8.45e-28
Time:                        06:48:50   Log-Likelihood:                 128.17
No. Observations:                  21   AIC:                            -248.3
Df Residuals:                      17   BIC:                            -244.2
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept        0.0045      0.001      4.998      0.000       0.003       0.006
totpop_tj_s1     0.9430      0.009    106.272      0.000       0.924       0.962
gdpr_tj_diff  6.071e-06   3.77e-07     16.100      0.000    5.28e-06    6.87e-06
d_totpop_tj      0.0012      0.000      8.397      0.000       0.001       0.001
==============================================================================
Omnibus:                        0.183   Durbin-Watson:                   1.662
Prob(Omnibus):                  0.912   Jarque-Bera (JB):                0.327
Skew:                           0.182   Prob(JB):                        0.849
Kurtosis:                       2.509   Cond. No.                     6.75e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.75e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq169 : popu_tj 行为方程 ok
popu_tj  = @coef(1)  + @coef(2) * popu_tj(-1) 

In [28]:
test1 = df_ev[["popu_tj" ]]
test1["popu_tj_s1"] = test1["popu_tj"].shift(1)
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,popu_tj,popu_tj_s1
2006,0.081410,0.07834
2007,0.085089,0.08141


In [29]:
results = sm.ols(formula = "popu_tj ~  popu_tj_s1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                popu_tj   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     909.6
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           6.29e-12
Time:                        06:55:21   Log-Likelihood:                 63.132
No. Observations:                  13   AIC:                            -122.3
Df Residuals:                      11   BIC:                            -121.1
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0110      0.003      3.274      0.007       0.004       0.018
popu_tj_s1     0.9340      0.031     30.160      0.000       0.866       1.002
==============================================================================
Omnibus:                        1.875   Durbin-Watson:                   0.548
Prob(Omnibus):                  0.392   Jarque-Bera (JB):                0.899
Skew:                          -0.119   Prob(JB):                        0.638
Kurtosis:                       1.734   Cond. No.                         55.2
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq170 : rpi_tj 行为方程 ok
rpi_tj  = @coef(1)  + @coef(2) * d(cpi_tj)  + @coef(3) * rpi_tj(-1) 

In [32]:
test1 = df_ev[["rpi_tj","cpi_tj"]]
test1["cpi_tj_diff"] = test1["cpi_tj"] - test1["cpi_tj"].shift(1)
test1["rpi_tj_s1"] = test1["rpi_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,rpi_tj,cpi_tj,cpi_tj_diff,rpi_tj_s1
1997,92.489105,73.412284,2.207353,91.846182
1998,89.344476,73.045223,-0.367061,92.489105


In [33]:
results = sm.ols(formula = "rpi_tj ~ cpi_tj_diff + rpi_tj_s1  ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 rpi_tj   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     1012.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           5.04e-20
Time:                        07:01:53   Log-Likelihood:                -24.148
No. Observations:                  22   AIC:                             54.30
Df Residuals:                      19   BIC:                             57.57
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      -0.9749      2.142     -0.455      0.654      -5.457       3.507
cpi_tj_diff     1.2960      0.115     11.300      0.000       1.056       1.536
rpi_tj_s1       0.9934      0.024     41.482      0.000       0.943       1.044
==============================================================================
Omnibus:                        0.721   Durbin-Watson:                   1.566
Prob(Omnibus):                  0.697   Jarque-Bera (JB):                0.118
Skew:                           0.163   Prob(JB):                        0.943
Kurtosis:                       3.150   Cond. No.                     1.16e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.16e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq171 : cpi_tj 行为方程  ok
cpi_tj  = @coef(1)  + @coef(2) * cpi_tj(-1)  + @coef(3) * gdpd_tj  + @coef(4) * d_cpi_tj 

In [36]:
test1 = df_ev[["cpi_tj","gdpd_tj" ,"d_cpi_tj"]]
test1["cpi_tj_s1"] = test1["cpi_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,cpi_tj,gdpd_tj,d_cpi_tj,cpi_tj_s1
1997,73.412284,74.109999,1,71.204931
1998,73.045223,73.691574,0,73.412284


In [38]:
results = sm.ols(formula = "cpi_tj ~ cpi_tj_s1 + gdpd_tj + d_cpi_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 cpi_tj   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     2301.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.91e-23
Time:                        07:07:28   Log-Likelihood:                -20.112
No. Observations:                  22   AIC:                             48.22
Df Residuals:                      18   BIC:                             52.59
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -6.2589      1.160     -5.395      0.000      -8.696      -3.822
cpi_tj_s1      0.9642      0.020     47.505      0.000       0.922       1.007
gdpd_tj        0.1201      0.019      6.251      0.000       0.080       0.160
d_cpi_tj       1.6273      0.275      5.910      0.000       1.049       2.206
==============================================================================
Omnibus:                        1.240   Durbin-Watson:                   2.144
Prob(Omnibus):                  0.538   Jarque-Bera (JB):                1.027
Skew:                           0.313   Prob(JB):                        0.599
Kurtosis:                       2.147   Cond. No.                     1.01e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq172 ：gdp1d_tj 行为方程  ok
gdp1d_tj  = @coef(1)  + @coef(2) * d(rpi_tj)  + @coef(3) * gdp1d_tj(-1)  + @coef(4) * cpi_tj  + @coef(5) * d_gdp1d_tj

In [39]:
test1 = df_ev[["gdp1d_tj","rpi_tj","cpi_tj","d_gdp1d_tj"]]
test1["rpi_tj_diff"] = test1["rpi_tj"] - test1["rpi_tj"].shift(1)
test1["gdp1d_tj_s1"] = test1["gdp1d_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,gdp1d_tj,rpi_tj,cpi_tj,d_gdp1d_tj,rpi_tj_diff,gdp1d_tj_s1
1997,87.622905,92.489105,73.412284,0.0,0.642923,91.529518
1998,88.657887,89.344476,73.045223,0.5,-3.144630,87.622905


In [40]:
results = sm.ols(formula = "gdp1d_tj ~ rpi_tj_diff + gdp1d_tj_s1 + cpi_tj + d_gdp1d_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               gdp1d_tj   R-squared:                       0.980
Model:                            OLS   Adj. R-squared:                  0.975
Method:                 Least Squares   F-statistic:                     206.6
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           3.61e-14
Time:                        07:12:50   Log-Likelihood:                -29.013
No. Observations:                  22   AIC:                             68.03
Df Residuals:                      17   BIC:                             73.48
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept      21.9992      4.298      5.119      0.000      12.932      31.067
rpi_tj_diff     0.4085      0.130      3.136      0.006       0.134       0.683
gdp1d_tj_s1     0.5134      0.073      7.061      0.000       0.360       0.667
cpi_tj          0.2660      0.040      6.587      0.000       0.181       0.351
d_gdp1d_tj      4.7254      0.407     11.597      0.000       3.866       5.585
==============================================================================
Omnibus:                        8.337   Durbin-Watson:                   1.706
Prob(Omnibus):                  0.015   Jarque-Bera (JB):                2.036
Skew:                          -0.157   Prob(JB):                        0.361
Kurtosis:                       1.543   Cond. No.                     2.46e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.46e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq173 :gdp2d_tj  行为方程  ok
gdp2d_tj  = @coef(1)  + @coef(2) * ppi_tj  + @coef(3) * gdp2d_tj(-1)  + @coef(4) * d(faipi_tj)  + @coef(5) * d_gdp2d_tj

In [43]:
test1 = df_ev[["gdp2d_tj","ppi_tj","faipi_tj","d_gdp2d_tj"]]
test1["gdp2d_tj_s1"] = test1["gdp2d_tj"].shift(1)
test1["faipi_tj_diff"] = test1["faipi_tj"] - test1["faipi_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,gdp2d_tj,ppi_tj,faipi_tj,d_gdp2d_tj,gdp2d_tj_s1,faipi_tj_diff
1997,107.814632,123.600414,77.384583,0.0,108.114214,0.614163
1998,104.927400,117.049592,76.533353,0.0,107.814632,-0.851230


In [44]:
results = sm.ols(formula = "gdp2d_tj ~ ppi_tj + gdp2d_tj_s1 + faipi_tj_diff + d_gdp2d_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               gdp2d_tj   R-squared:                       0.977
Model:                            OLS   Adj. R-squared:                  0.972
Method:                 Least Squares   F-statistic:                     180.7
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.09e-13
Time:                        07:21:45   Log-Likelihood:                -41.399
No. Observations:                  22   AIC:                             92.80
Df Residuals:                      17   BIC:                             98.25
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
=================================================================================
                    coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------
Intercept       -11.4974      8.282     -1.388      0.183     -28.970       5.975
ppi_tj            0.2849      0.080      3.553      0.002       0.116       0.454
gdp2d_tj_s1       0.7857      0.051     15.294      0.000       0.677       0.894
faipi_tj_diff     1.3002      0.177      7.343      0.000       0.927       1.674
d_gdp2d_tj        4.4260      0.648      6.833      0.000       3.059       5.793
==============================================================================
Omnibus:                        2.463   Durbin-Watson:                   1.808
Prob(Omnibus):                  0.292   Jarque-Bera (JB):                1.733
Skew:                          -0.684   Prob(JB):                        0.420
Kurtosis:                       2.852   Cond. No.                     3.38e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.38e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq174 : gdp3d_tj 行为方程 ok
gdp3d_tj  = @coef(1)  + @coef(2) * gdp1d_tj  + @coef(3) * gdp2d_tj  + @coef(4) * gdp3d_tj(-1)

In [45]:
test1 = df_ev[["gdp3d_tj","gdp2d_tj","gdp1d_tj"]]
test1["gdp3d_tj_s1"] = test1["gdp3d_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,gdp3d_tj,gdp2d_tj,gdp1d_tj,gdp3d_tj_s1
1997,51.906286,107.814632,87.622905,50.294340
1998,53.960349,104.927400,88.657887,51.906286


In [46]:
results = sm.ols(formula = "gdp3d_tj ~ gdp1d_tj + gdp2d_tj + gdp3d_tj_s1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               gdp3d_tj   R-squared:                       0.989
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     548.8
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           7.10e-18
Time:                        07:26:28   Log-Likelihood:                -46.169
No. Observations:                  22   AIC:                             100.3
Df Residuals:                      18   BIC:                             104.7
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===============================================================================
                  coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------
Intercept     -30.7989     11.321     -2.721      0.014     -54.582      -7.015
gdp1d_tj        0.2662      0.153      1.744      0.098      -0.054       0.587
gdp2d_tj        0.1288      0.047      2.750      0.013       0.030       0.227
gdp3d_tj_s1     0.9303      0.053     17.712      0.000       0.820       1.041
==============================================================================
Omnibus:                        2.242   Durbin-Watson:                   1.916
Prob(Omnibus):                  0.326   Jarque-Bera (JB):                1.293
Skew:                           0.592   Prob(JB):                        0.524
Kurtosis:                       3.086   Cond. No.                     3.92e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.92e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq175 gdpd_tj 行为方程 ok
gdpd_tj  = @coef(1)  + @coef(2) * gdp2d_tj  + @coef(3) * gdp3d_tj

In [47]:
test1 = df_ev[["gdpd_tj","gdp2d_tj","gdp3d_tj"]]
test1 = test1[(test1.index>1995) & (test1.index <2019)]
test1.head(2)

,gdpd_tj,gdp2d_tj,gdp3d_tj
1996,73.693486,108.114214,50.294340
1997,74.109999,107.814632,51.906286


In [48]:
results = sm.ols(formula = "gdpd_tj ~  gdp2d_tj + gdp3d_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                gdpd_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                 1.250e+04
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.06e-31
Time:                        07:29:09   Log-Likelihood:                -7.8154
No. Observations:                  23   AIC:                             21.63
Df Residuals:                      20   BIC:                             25.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept    -11.0982      0.922    -12.040      0.000     -13.021      -9.175
gdp2d_tj       0.4994      0.008     66.160      0.000       0.484       0.515
gdp3d_tj       0.6126      0.004    153.510      0.000       0.604       0.621
==============================================================================
Omnibus:                        1.309   Durbin-Watson:                   1.130
Prob(Omnibus):                  0.520   Jarque-Bera (JB):                0.884
Skew:                           0.060   Prob(JB):                        0.643
Kurtosis:                       2.047   Cond. No.                     1.61e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.61e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq176 : ppi_tj 行为方程 ok
log(ppi_tj)  = @coef(1)  + @coef(2) * (log(rpi_tj) - log(rpi_tj(-1)))  + @coef(3) * log(ppi_tj(-1))  + @coef(4) * d_ppi_tj

In [49]:
test1 = df_ev[["ppi_tj","rpi_tj","d_ppi_tj"]]
test1["rpi_tj_diff"] = np.log(test1["rpi_tj"])- np.log(test1["rpi_tj"].shift(1))
test1["ppi_tj_s1"] = test1["ppi_tj"].shift(1)
test1 = test1[(test1.index>1996) & (test1.index <2019)]
test1.head(2)

,ppi_tj,rpi_tj,d_ppi_tj,rpi_tj_diff,ppi_tj_s1
1997,123.600414,92.489105,0.0,0.006976,125.737959
1998,117.049592,89.344476,0.0,-0.034591,123.600414


In [50]:
results = sm.ols(formula = "np.log(ppi_tj) ~ rpi_tj_diff + np.log(ppi_tj_s1)  + d_ppi_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:         np.log(ppi_tj)   R-squared:                       0.946
Model:                            OLS   Adj. R-squared:                  0.938
Method:                 Least Squares   F-statistic:                     106.0
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.25e-11
Time:                        07:30:54   Log-Likelihood:                 64.064
No. Observations:                  22   AIC:                            -120.1
Df Residuals:                      18   BIC:                            -115.8
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             2.2057      0.246      8.955      0.000       1.688       2.723
rpi_tj_diff           0.5181      0.141      3.684      0.002       0.223       0.814
np.log(ppi_tj_s1)     0.5347      0.052     10.286      0.000       0.426       0.644
d_ppi_tj              0.0657      0.006     10.307      0.000       0.052       0.079
==============================================================================
Omnibus:                        0.029   Durbin-Watson:                   2.216
Prob(Omnibus):                  0.985   Jarque-Bera (JB):                0.086
Skew:                           0.015   Prob(JB):                        0.958
Kurtosis:                       2.695   Cond. No.                         393.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq177 ：faipi_tj 行为方程 ok arma
log(faipi_tj)  = @coef(1)  + @coef(2) * d(ppi_tj)  + @coef(3) * log(faipi_tj(-1))  + [ar(2) = @coef(4) , estsmpl = "1999 2018"]

In [51]:
test1 = df_ev[["faipi_tj","ppi_tj"]]
test1["ppi_tj_diff"] = test1["ppi_tj"] - test1["ppi_tj"].shift(1)
test1["faipi_tj_s1"] = np.log(test1["faipi_tj"].shift(1))
test1 = test1[(test1.index>1998) & (test1.index <2019)]
test1.head(2)

,faipi_tj,ppi_tj,ppi_tj_diff,faipi_tj_s1
1999,75.921086,112.835807,-4.213785,4.337727
2000,75.845165,115.995209,3.159403,4.329694


In [53]:
ar = (0,1)
ma = 0
mod = SARIMAX(endog = np.log(test1["faipi_tj"]) , exog = test1[["ppi_tj_diff","faipi_tj_s1"]],order =(ar,0,ma),enforce_invertibility= False , trend = 'c', enforce_stationarity = False)
results = mod.fit()
results.summary() #results.summary().tables[1]

<class 'statsmodels.iolib.summary.Summary'>
"""
                               SARIMAX Results                                
==============================================================================
Dep. Variable:               faipi_tj   No. Observations:                   20
Model:             SARIMAX([2], 0, 0)   Log Likelihood                  49.713
Date:                Mon, 13 Jul 2020   AIC                            -89.426
Time:                        07:38:23   BIC                            -84.975
Sample:                             0   HQIC                           -88.813
                                 - 20                                         
Covariance Type:                  opg                                         
===============================================================================
                  coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------
intercept       0.0253      0.184      0.137      0.891      -0.336       0.386
ppi_tj_diff     0.0047      0.001      6.096      0.000       0.003       0.006
faipi_tj_s1     1.0006      0.029     34.663      0.000       0.944       1.057
ar.L2          -0.4369      0.351     -1.244      0.213      -1.125       0.251
sigma2          0.0002      0.000      1.806      0.071   -1.99e-05       0.000
===================================================================================
Ljung-Box (Q):                       17.15   Jarque-Bera (JB):                 4.17
Prob(Q):                              0.44   Prob(JB):                         0.12
Heteroskedasticity (H):               0.74   Skew:                             1.17
Prob(H) (two-sided):                  0.73   Kurtosis:                         3.33
===================================================================================

Warnings:
[1] Covariance matrix calculated using the outer product of gradients (complex-step).
"""

## Eq178 : gdp3_tspar_tj 定义方程
gdp3_tspar_tj  = gdp3_tspan_tj  / pgdp3_tspad_tj  * 100

In [54]:
test1 = df_ev[["gdp3_tspar_tj","gdp3_tspan_tj","pgdp3_tspad_tj"]]
test1["预测"] = test1["gdp3_tspan_tj"] / test1["pgdp3_tspad_tj"] *100
test1.tail(4)

,gdp3_tspar_tj,gdp3_tspan_tj,pgdp3_tspad_tj,预测
2017,813.016279,780.40,95.988238,813.016279
2018,838.219784,816.33,97.388539,838.219784
2019,NaN,NaN,98.809268,NaN
2020,NaN,NaN,NaN,NaN


## Eq179 :gdp3_tspan_tj  行为方程 ok

gdp3_tspan_tj  = @coef(1)  + @coef(2) * gdp3_tratp_tj  + @coef(3) * gdp3_trapt_tj(-1)  + @coef(4) * d_gdp3_tspan_tj

In [56]:
test1 = df_ev[["gdp3_tspan_tj","gdp3_tratp_tj","gdp3_trapt_tj","d_gdp3_tspan_tj"]]
test1["gdp3_trapt_tj_s1"] = test1["gdp3_trapt_tj"].shift(1)
test1 = test1[(test1.index>2004) & (test1.index <2019)]
test1.head(2)

,gdp3_tspan_tj,gdp3_tratp_tj,gdp3_trapt_tj,d_gdp3_tspan_tj,gdp3_trapt_tj_s1
2005,227.16,142.10,176.3477,0.0,146.7836
2006,252.86,163.41,227.7902,0.0,176.3477


In [57]:
results = sm.ols(formula = " gdp3_tspan_tj~ gdp3_tratp_tj + gdp3_trapt_tj_s1 + d_gdp3_tspan_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp3_tspan_tj   R-squared:                       0.991
Model:                            OLS   Adj. R-squared:                  0.988
Method:                 Least Squares   F-statistic:                     364.8
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           1.64e-10
Time:                        07:47:22   Log-Likelihood:                -60.683
No. Observations:                  14   AIC:                             129.4
Df Residuals:                      10   BIC:                             131.9
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          -34.6064     23.401     -1.479      0.170     -86.747      17.534
gdp3_tratp_tj        1.3443      0.044     30.242      0.000       1.245       1.443
gdp3_trapt_tj_s1     0.4067      0.070      5.836      0.000       0.251       0.562
d_gdp3_tspan_tj    118.8735     17.274      6.882      0.000      80.385     157.362
==============================================================================
Omnibus:                        0.070   Durbin-Watson:                   1.555
Prob(Omnibus):                  0.966   Jarque-Bera (JB):                0.130
Skew:                           0.097   Prob(JB):                        0.937
Kurtosis:                       2.570   Cond. No.                     2.00e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large,  2e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq180 gdp3_wrar_tj 定义方程式 ok
gdp3_wrar_tj  = gdp3_wran_tj  / pgdp3_wrad_tj  * 100

In [58]:
test1 = df_ev[["gdp3_wrar_tj","gdp3_wran_tj","pgdp3_wrad_tj"]]
test1["预测"] = test1["gdp3_wran_tj"] / test1["pgdp3_wrad_tj"] *100
test1.tail(4)

,gdp3_wrar_tj,gdp3_wran_tj,pgdp3_wrad_tj,预测
2017,2286.568254,2306.98,100.890000,2286.629002
2018,2304.860800,2361.45,102.460000,2304.753074
2019,NaN,NaN,104.054432,NaN
2020,NaN,NaN,NaN,NaN


## Eq181 : gdp3_wran_tj 行为方程式  ok
gdp3_wran_tj  = @coef(1)  + @coef(2) * consn_tj  + @coef(3) * gdp3_wran_tj(-1)  + @coef(4) * d_gdp3_wran_tj


In [60]:
test1 = df_ev[["gdp3_wran_tj","consn_tj","d_gdp3_wran_tj" ]]
test1["gdp3_wran_tj_s1"] = test1["gdp3_wran_tj"].shift(1)
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,gdp3_wran_tj,consn_tj,d_gdp3_wran_tj,gdp3_wran_tj_s1
2006,468.12,1356.79,0.0,436.14
2007,498.61,1603.74,0.0,468.12


In [61]:
results = sm.ols(formula = " gdp3_wran_tj~ consn_tj + gdp3_wran_tj_s1 + d_gdp3_wran_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_wran_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     1250.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           4.15e-12
Time:                        07:51:44   Log-Likelihood:                -63.901
No. Observations:                  13   AIC:                             135.8
Df Residuals:                       9   BIC:                             138.1
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         -82.1915     46.865     -1.754      0.113    -188.207      23.825
consn_tj            0.2905      0.052      5.584      0.000       0.173       0.408
gdp3_wran_tj_s1     0.3423      0.117      2.937      0.017       0.079       0.606
d_gdp3_wran_tj    173.3212     34.017      5.095      0.001      96.370     250.273
==============================================================================
Omnibus:                        3.215   Durbin-Watson:                   2.689
Prob(Omnibus):                  0.200   Jarque-Bera (JB):                1.130
Skew:                          -0.138   Prob(JB):                        0.568
Kurtosis:                       1.582   Cond. No.                     1.85e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.85e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq182 : gdp3_fiar_tj 定义方程 ok
gdp3_fiar_tj  = gdp3_fian_tj  / pgdp3_fiad_tj  * 100

In [112]:
test1 = df_ev[["gdp3_fiar_tj","gdp3_fian_tj","pgdp3_fiad_tj"]]
test1["预测"] = test1["gdp3_fian_tj"] / test1["pgdp3_fiad_tj"] *100
test1.tail(4)

,gdp3_fiar_bj,gdp3_fian_bj,pgdp3_fiad_bj,预测
2017,4587.84527,4655.4,101.472472,4587.845270
2018,4918.17013,5084.6,103.383980,4918.170130
2019,NaN,6544.8,105.331496,6213.526127
2020,NaN,NaN,NaN,NaN


## Eq183 : gdp3_fian_tj 行为方程
gdp3_fian_tj  = @coef(1)  + @coef(2) * depn_tj  + @coef(3) * gdp3_fian_tj(-1)

In [62]:
test1 = df_ev[["gdp3_fian_tj","depn_tj"]]
test1["gdp3_fian_tj_s1"] = test1["gdp3_fian_tj"].shift(1)
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,gdp3_fian_bj,depn_bj,gdp3_fian_bj_s1
2006,982.4,31313.8,840.2
2007,1302.8,35369.7,982.4


In [118]:
results = sm.ols(formula = " gdp3_fian_tj~ depn_tj + gdp3_fian_tj_s1 ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_fian_bj   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     1982.
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           7.56e-16
Time:                        04:34:31   Log-Likelihood:                -86.414
No. Observations:                  15   AIC:                             178.8
Df Residuals:                      12   BIC:                             181.0
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          62.0117     51.260      1.210      0.250     -49.674     173.698
depn_bj             0.0095      0.005      2.098      0.058      -0.000       0.019
gdp3_fian_bj_s1     0.7751      0.147      5.280      0.000       0.455       1.095
==============================================================================
Omnibus:                        0.018   Durbin-Watson:                   1.612
Prob(Omnibus):                  0.991   Jarque-Bera (JB):                0.195
Skew:                           0.065   Prob(JB):                        0.907
Kurtosis:                       2.457   Cond. No.                     2.03e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.03e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq184 : gdp3_rear_tj 定义方程  ok
gdp3_rear_tj  = gdp3_rean_tj  / pgdp3_read_tj  * 100

In [63]:
test1 = df_ev[["gdp3_rear_tj","gdp3_rean_tj","pgdp3_read_tj"]]
test1["预测"] = test1["gdp3_rean_tj"] / test1["pgdp3_read_tj"] *100
test1.tail(4)

,gdp3_rear_tj,gdp3_rean_tj,pgdp3_read_tj,预测
2017,514.050562,783.27,152.372171,514.050562
2018,546.468165,724.39,132.558500,546.468165
2019,NaN,NaN,115.321293,NaN
2020,NaN,NaN,NaN,NaN


## Eq185 : gdp3_rean_tj 行为方程  ok
gdp3_rean_tj  = @coef(1)  + @coef(2) * inv_rean_tj  + @coef(3) * gdp3_rean_tj(-1) 

In [122]:
test1 = df_ev[["gdp3_rean_tj","inv_rean_tj"]]
test1["gdp3_rean_tj_s1"] = test1["gdp3_rean_tj"].shift(1)
test1 = test1[(test1.index>2005) & (test1.index <2019)]
test1.head(2)

,inv_rean_bj,gdp3_rean_bj,d_gdp3_rean_bj,gdp3_rean_bj_s1
2003,1223.61,341.9,0.0,298.0
2004,1491.84,436.1,0.0,341.9


In [124]:
results = sm.ols(formula = " gdp3_rean_tj~ inv_rean_tj + gdp3_rean_tj_s1  ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_rean_bj   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     458.7
Date:                Fri, 10 Jul 2020   Prob (F-statistic):           7.86e-12
Time:                        04:46:15   Log-Likelihood:                -75.723
No. Observations:                  15   AIC:                             159.4
Df Residuals:                      11   BIC:                             162.3
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept          34.6942     32.322      1.073      0.306     -36.445     105.834
inv_rean_bj         0.0953      0.042      2.274      0.044       0.003       0.188
gdp3_rean_bj_s1     0.7694      0.119      6.461      0.000       0.507       1.031
d_gdp3_rean_bj    170.0664     30.492      5.577      0.000     102.955     237.178
==============================================================================
Omnibus:                        2.453   Durbin-Watson:                   2.079
Prob(Omnibus):                  0.293   Jarque-Bera (JB):                1.398
Skew:                           0.465   Prob(JB):                        0.497
Kurtosis:                       1.830   Cond. No.                     9.74e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq 186 : gdp3_infr_tj 定义方程 ok
gdp3_infr_tj  = gdp3_infn_tj  / pgdp3_infd_tj  * 100

In [125]:
test1 = df_ev[["gdp3_infr_tj","gdp3_infn_tj","pgdp3_infd_tj"]]
test1["预测"] = test1["gdp3_infn_tj"] / test1["pgdp3_infd_tj"] *100
test1.tail(4)

,gdp3_infr_bj,gdp3_infn_bj,pgdp3_infd_bj,预测
2017,3098.746681,3229.0,104.203419,3098.746681
2018,3687.508550,3859.0,104.650605,3687.508550
2019,NaN,NaN,105.099709,NaN
2020,NaN,NaN,NaN,NaN


## Eq 187 : gdp3_infn_tj 行为方程 ok

gdp3_infn_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj

In [64]:
test1 = df_ev[["gdp3_infn_tj","gdp2_tran_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp3_infn_tj~ gdp2_tran_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_infn_tj   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.987
Method:                 Least Squares   F-statistic:                     296.2
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           0.000428
Time:                        07:59:19   Log-Likelihood:                -20.619
No. Observations:                   5   AIC:                             45.24
Df Residuals:                       3   BIC:                             44.46
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     -348.2137     44.246     -7.870      0.004    -489.024    -207.403
gdp2_tran_tj     0.4234      0.025     17.209      0.000       0.345       0.502
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.898
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.428
Skew:                          -0.023   Prob(JB):                        0.807
Kurtosis:                       1.567   Cond. No.                     9.22e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq188 : gdp3_renr_tj 定义方程
gdp3_renr_tj  = gdp3_renn_tj  / pgdp3_rend_tj  * 100

In [65]:
test1 = df_ev[["gdp3_renr_tj","gdp3_renn_tj","pgdp3_rend_tj"]]
test1["预测"] = test1["gdp3_renn_tj"] / test1["pgdp3_rend_tj"] *100
test1.tail(4)

,gdp3_renr_tj,gdp3_renn_tj,pgdp3_rend_tj,预测
2017,1002.768672,997.30,99.454643,1002.768673
2018,1061.932024,1001.48,94.307355,1061.932024
2019,NaN,NaN,89.426465,NaN
2020,NaN,NaN,NaN,NaN


## Eq189 : gdp3_renn_tj 行为方程 ok
gdp3_renn_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj  + @coef(3) * d_gdp3_renn_tj

In [67]:
test1 = df_ev[["gdp3_renn_tj","gdp2_tran_tj","d_gdp3_renn_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp3_renn_tj~ gdp2_tran_tj + d_gdp3_renn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_renn_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.999
Method:                 Least Squares   F-statistic:                     1337.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           0.000747
Time:                        08:03:05   Log-Likelihood:                -14.081
No. Observations:                   5   AIC:                             34.16
Df Residuals:                       2   BIC:                             32.99
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         55.7477     15.764      3.536      0.071     -12.079     123.574
gdp2_tran_tj       0.4488      0.009     51.710      0.000       0.411       0.486
d_gdp3_renn_tj    80.2479      4.502     17.827      0.003      60.879      99.616
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.395
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.645
Skew:                           0.462   Prob(JB):                        0.724
Kurtosis:                       1.503   Cond. No.                     9.97e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.97e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq190 :gdp3_sicr_tj 定义方程
gdp3_sicr_tj  = gdp3_sicn_tj  / pgdp3_sicd_tj  * 100

In [68]:
test1 = df_ev[["gdp3_sicr_tj","gdp3_sicn_tj","pgdp3_sicd_tj"]]
test1["预测"] = test1["gdp3_sicn_tj"] / test1["pgdp3_sicd_tj"] *100
test1.tail(4)

,gdp3_sicr_tj,gdp3_sicn_tj,pgdp3_sicd_tj,预测
2017,879.654216,996.39,113.270645,879.654216
2018,968.499292,1005.18,103.787376,968.499292
2019,NaN,NaN,95.098067,NaN
2020,NaN,NaN,NaN,NaN


## Eq191 : gdp3_sicn_tj 行为方程 ok
gdp3_sicn_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj  + @coef(3) * d_gdp3_sicn_tj

In [69]:
test1 = df_ev[["gdp3_sicn_tj","gdp2_tran_tj","d_gdp3_sicn_tj"]]
test1["gdp3_sicn_tj_s1"] = test1["gdp3_sicn_tj"].shift(1)
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp3_sicn_tj~ gdp2_tran_tj  +  d_gdp3_sicn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp3_sicn_tj   R-squared:                       0.988
Model:                            OLS   Adj. R-squared:                  0.977
Method:                 Least Squares   F-statistic:                     84.52
Date:                Mon, 13 Jul 2020   Prob (F-statistic):             0.0117
Time:                        08:19:05   Log-Likelihood:                -20.957
No. Observations:                   5   AIC:                             47.91
Df Residuals:                       2   BIC:                             46.74
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         54.0287     63.203      0.855      0.483    -217.911     325.969
gdp2_tran_tj       0.4530      0.035     12.977      0.006       0.303       0.603
d_gdp3_sicn_tj    81.5159     19.233      4.238      0.051      -1.235     164.267
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.929
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.511
Skew:                           0.737   Prob(JB):                        0.774
Kurtosis:                       2.469   Cond. No.                     1.01e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.01e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq192 : gdp2_tecn_tj1  行为方程 ok
gdp2_tecn_tj1  = @coef(1)  + @coef(2) * tta_tj  + @coef(3) * d_gdp2_tecn_tj_1

In [70]:
test1 = df_ev[["gdp2_tecn_tj1","tta_tj","d_gdp2_tecn_tj_1"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_tecn_tj1~ tta_tj + d_gdp2_tecn_tj_1",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp2_tecn_tj1   R-squared:                       0.993
Model:                            OLS   Adj. R-squared:                  0.986
Method:                 Least Squares   F-statistic:                     143.7
Date:                Mon, 13 Jul 2020   Prob (F-statistic):            0.00691
Time:                        08:21:16   Log-Likelihood:                -18.646
No. Observations:                   5   AIC:                             43.29
Df Residuals:                       2   BIC:                             42.12
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          614.2611     34.694     17.705      0.003     464.984     763.538
tta_tj               1.3037      0.078     16.804      0.004       0.970       1.638
d_gdp2_tecn_tj_1    30.3818      8.928      3.403      0.077      -8.034      68.798
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.214
Prob(Omnibus):                    nan   Jarque-Bera (JB):                1.708
Skew:                           1.430   Prob(JB):                        0.426
Kurtosis:                       3.153   Cond. No.                     2.17e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.17e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq193 : gdp2_tecn_tj2行为方程 ok
gdp2_tecn_tj2  = @coef(1)  + @coef(2) * lse_tj  + @coef(3) * d_gdp2_tecn_tj_2

In [72]:
test1 = df_ev[["gdp2_tecn_tj2","lse_tj","d_gdp2_tecn_tj_2"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_tecn_tj2 ~ lse_tj + d_gdp2_tecn_tj_2",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp2_tecn_tj2   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     207.8
Date:                Mon, 13 Jul 2020   Prob (F-statistic):            0.00479
Time:                        08:26:16   Log-Likelihood:                -17.849
No. Observations:                   5   AIC:                             41.70
Df Residuals:                       2   BIC:                             40.53
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          605.2506     28.826     20.996      0.002     481.221     729.280
lse_tj               0.0045      0.000     20.372      0.002       0.004       0.005
d_gdp2_tecn_tj_2    67.5075      9.303      7.257      0.018      27.480     107.535
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.555
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.412
Skew:                           0.490   Prob(JB):                        0.814
Kurtosis:                       1.992   Cond. No.                     6.16e+05
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.16e+05. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq 194 ：gdp2_tecn_tj 定义方程
gdp2_tecn_tj  = (gdp2_tecn_tj1  + gdp2_tecn_tj2)  / 2

In [79]:
test1 = df_ev[["gdp2_tecn_tj","gdp2_tecn_tj1","gdp2_tecn_tj2"]]
test1["预测"] = (test1["gdp2_tecn_tj1"] + test1["gdp2_tecn_tj2"])/2
test1.tail(4)

,gdp2_tecn_tj,gdp2_tecn_tj1,gdp2_tecn_tj2,预测
2017,1308.576,1303.573,1313.578,1308.5755
2018,1298.540,1298.540,1298.540,1298.5400
2019,1412.020,1412.020,1412.020,1412.0200
2020,NaN,NaN,NaN,NaN


## Eq 195 gdp2_medn_tj 行为方程

gdp2_medn_tj  = @coef(1)  + @coef(2) * gdp2_tecn_tj  + @coef(3) * d_gdp2_medn_tj

In [80]:
test1 = df_ev[["gdp2_medn_tj","gdp2_tecn_tj","d_gdp2_medn_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_medn_tj~ gdp2_tecn_tj + d_gdp2_medn_tj ",data = test1 ).fit()
results.summary()


<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_medn_tj   R-squared:                       0.990
Model:                            OLS   Adj. R-squared:                  0.980
Method:                 Least Squares   F-statistic:                     98.41
Date:                Mon, 13 Jul 2020   Prob (F-statistic):             0.0101
Time:                        09:10:17   Log-Likelihood:                -18.242
No. Observations:                   5   AIC:                             42.48
Df Residuals:                       2   BIC:                             41.31
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       -324.6524     69.984     -4.639      0.043    -625.771     -23.534
gdp2_tecn_tj       0.5105      0.059      8.644      0.013       0.256       0.765
d_gdp2_medn_tj    57.0754      8.855      6.445      0.023      18.973      95.177
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.915
Prob(Omnibus):                    nan   Jarque-Bera (JB):                1.156
Skew:                          -1.177   Prob(JB):                        0.561
Kurtosis:                       2.926   Cond. No.                     1.27e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.27e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

##  Eq196 : tta_tj 行为  北京和天津数据有影响
tta_tj  = @coef(1)  + @coef(2) * tta_f_bj  + @coef(3) * tta_tj(-1)  + @coef(4) * d_tta_tj

In [83]:
test1 = df_ev[["tta_tj","tta_f_bj","d_tta_tj"]]
test1["tta_tj_s1"] = test1["tta_tj"].shift(1)
test1 = test1[(test1.index>2006) & (test1.index <2019)]
results = sm.ols(formula = " tta_tj~ tta_f_bj + tta_tj_s1 +d_tta_tj ",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                 tta_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.997
Method:                 Least Squares   F-statistic:                     1222.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           5.53e-11
Time:                        09:15:11   Log-Likelihood:                -42.177
No. Observations:                  12   AIC:                             92.35
Df Residuals:                       8   BIC:                             94.29
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -1.2314      6.166     -0.200      0.847     -15.451      12.988
tta_f_bj       0.0564      0.011      5.221      0.001       0.031       0.081
tta_tj_s1      0.7965      0.056     14.221      0.000       0.667       0.926
d_tta_tj      91.2760     10.636      8.582      0.000      66.749     115.803
==============================================================================
Omnibus:                        4.519   Durbin-Watson:                   3.130
Prob(Omnibus):                  0.104   Jarque-Bera (JB):                1.773
Skew:                           0.882   Prob(JB):                        0.412
Kurtosis:                       3.661   Cond. No.                     6.02e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.02e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq197 : gdp2_airn_tj 行为方程 ok
gdp2_airn_tj  = @coef(1)  + @coef(2) * d(gdp2_tecn_tj)  + @coef(3) * d_gdp2_airn_tj

In [7]:
test1 = df_ev[["gdp2_airn_tj","gdp2_tecn_tj","d_gdp2_airn_tj" ,"gdp2_tecn_tj1","gdp2_tecn_tj2"]]
test1

,gdp2_airn_tj,gdp2_tecn_tj,d_gdp2_airn_tj,gdp2_tecn_tj1,gdp2_tecn_tj2
1996,NaN,NaN,0.0,NaN,NaN
1997,NaN,NaN,0.0,NaN,NaN
1998,NaN,NaN,0.0,NaN,NaN
1999,NaN,NaN,0.0,NaN,NaN
2000,NaN,NaN,0.0,NaN,NaN
2001,NaN,NaN,0.0,NaN,NaN
2002,NaN,NaN,0.0,NaN,NaN
2003,NaN,NaN,0.0,NaN,NaN
2004,NaN,NaN,0.0,NaN,NaN
2005,NaN,NaN,0.0,NaN,NaN


In [5]:
test1 = df_ev[["gdp2_airn_tj","gdp2_tecn_tj","d_gdp2_airn_tj"]]
test1["gdp2_tecn_tj_diff"] = test1["gdp2_tecn_tj"] - test1["gdp2_tecn_tj"].shift(1)
test1 = test1[(test1.index>2014) & (test1.index <2019)]
#联立改成了 ：test1 = test1[(test1.index>2014) & (test1.index <2018)]
results = sm.ols(formula = " gdp2_airn_tj ~ gdp2_tecn_tj_diff + d_gdp2_airn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_airn_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     343.2
Date:                Tue, 21 Jul 2020   Prob (F-statistic):             0.0381
Time:                        03:10:00   Log-Likelihood:                -10.019
No. Observations:                   4   AIC:                             26.04
Df Residuals:                       1   BIC:                             24.20
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            35.7677      4.809      7.438      0.085     -25.337      96.873
gdp2_tecn_tj_diff     0.7276      0.045     16.192      0.039       0.157       1.299
d_gdp2_airn_tj      131.7351      5.150     25.581      0.025      66.302     197.168
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.004
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.650
Skew:                           0.018   Prob(JB):                        0.722
Kurtosis:                       1.025   Cond. No.                         217.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq198 : gdp2_caln_tj 行为方程 ok
gdp2_caln_tj  = @coef(1)  + @coef(2) * gdp2_tecn_tj  + @coef(3) * d_gdp2_caln_tj

In [85]:
test1 = df_ev[["gdp2_caln_tj","gdp2_tecn_tj","d_gdp2_caln_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_caln_tj ~ gdp2_tecn_tj + d_gdp2_caln_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_caln_tj   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     202.3
Date:                Mon, 13 Jul 2020   Prob (F-statistic):            0.00492
Time:                        09:24:35   Log-Likelihood:                -13.891
No. Observations:                   5   AIC:                             33.78
Df Residuals:                       2   BIC:                             32.61
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        301.0202     26.740     11.257      0.008     185.968     416.072
gdp2_tecn_tj       0.2906      0.023     12.916      0.006       0.194       0.387
d_gdp2_caln_tj    57.4019      3.649     15.731      0.004      41.702      73.102
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.455
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.497
Skew:                          -0.012   Prob(JB):                        0.780
Kurtosis:                       1.455   Cond. No.                     1.15e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.15e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq199 : gdp2_mecn_tj 行为方程  ok
gdp2_mecn_tj  = @coef(1)  + @coef(2) * gdp2_tecn_tj  + @coef(3) * d_gdp2_mecn_tj

In [86]:
test1 = df_ev[["gdp2_mecn_tj","gdp2_tecn_tj","d_gdp2_mecn_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_mecn_tj ~ gdp2_tecn_tj + d_gdp2_mecn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_mecn_tj   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                     8301.
Date:                Mon, 13 Jul 2020   Prob (F-statistic):           0.000120
Time:                        09:31:16   Log-Likelihood:                 2.5050
No. Observations:                   5   AIC:                            0.9900
Df Residuals:                       2   BIC:                           -0.1817
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        -91.2491      1.045    -87.350      0.000     -95.744     -86.754
gdp2_tecn_tj       0.1144      0.001    128.829      0.000       0.111       0.118
d_gdp2_mecn_tj     6.4000      0.156     41.052      0.001       5.729       7.071
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.297
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.103
Skew:                          -0.195   Prob(JB):                        0.950
Kurtosis:                       2.414   Cond. No.                     1.20e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.2e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq200 : gdp2_emin_tj 行为方程 ok 北京+天津
gdp2_emin_tj  = @coef(1)  + @coef(2) * gdp_momn_bj  + @coef(3) * d_gdp2_emin_tj

In [6]:
test1 = df_ev[["gdp2_emin_tj","gdp_momn_bj","d_gdp2_emin_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_emin_tj ~  gdp_momn_bj + d_gdp2_emin_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_emin_tj   R-squared:                       0.985
Model:                            OLS   Adj. R-squared:                  0.969
Method:                 Least Squares   F-statistic:                     64.10
Date:                Tue, 14 Jul 2020   Prob (F-statistic):             0.0154
Time:                        01:30:35   Log-Likelihood:                -32.608
No. Observations:                   5   AIC:                             71.22
Df Residuals:                       2   BIC:                             70.04
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -2.376e+04   2363.257    -10.055      0.010   -3.39e+04   -1.36e+04
gdp_momn_bj       13.6501      1.222     11.172      0.008       8.393      18.907
d_gdp2_emin_tj  2659.7709    318.630      8.348      0.014    1288.818    4030.724
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.473
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.411
Skew:                          -0.497   Prob(JB):                        0.814
Kurtosis:                       2.008   Cond. No.                     4.02e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 4.02e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq201 : gdp2_tran_tj 行为方程 ok
gdp2_tran_tj  = @coef(1)  + @coef(2) * gdp2_tecn_tj  + @coef(3) * d_gdp2_tran_tj


In [7]:
test1 = df_ev[["gdp2_tran_tj","gdp2_tecn_tj","d_gdp2_tran_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_tran_tj ~  gdp2_tecn_tj + d_gdp2_tran_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_tran_tj   R-squared:                       0.995
Model:                            OLS   Adj. R-squared:                  0.989
Method:                 Least Squares   F-statistic:                     182.5
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00545
Time:                        01:32:22   Log-Likelihood:                -23.366
No. Observations:                   5   AIC:                             52.73
Df Residuals:                       2   BIC:                             51.56
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept      -1096.7409    180.830     -6.065      0.026   -1874.789    -318.692
gdp2_tecn_tj       2.4345      0.152     16.034      0.004       1.781       3.088
d_gdp2_tran_tj   172.5515     22.945      7.520      0.017      73.826     271.277
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.313
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.555
Skew:                          -0.054   Prob(JB):                        0.758
Kurtosis:                       1.371   Cond. No.                     1.17e+04
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.17e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq202 : gdp3_infwn_tj 行为方程 ok
gdp3_infwn_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj  + @coef(3) * d_gdp3_infwn_tj


In [8]:
test1 = df_ev[["gdp3_infwn_tj","gdp2_tran_tj","d_gdp3_infwn_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp3_infwn_tj ~  gdp2_tran_tj + d_gdp3_infwn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp3_infwn_tj   R-squared:                       0.999
Model:                            OLS   Adj. R-squared:                  0.998
Method:                 Least Squares   F-statistic:                     1142.
Date:                Tue, 14 Jul 2020   Prob (F-statistic):           0.000875
Time:                        01:36:57   Log-Likelihood:                -13.912
No. Observations:                   5   AIC:                             33.82
Df Residuals:                       2   BIC:                             32.65
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
===================================================================================
                      coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------
Intercept         147.1726     14.751      9.977      0.010      83.703     210.642
gdp2_tran_tj        0.2295      0.008     28.788      0.001       0.195       0.264
d_gdp3_infwn_tj   239.1186      7.185     33.278      0.001     208.202     270.035
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.063
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.551
Skew:                           0.437   Prob(JB):                        0.759
Kurtosis:                       1.628   Cond. No.                     9.71e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.71e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq203 : gdp2_hemin_tj 行为方程 ok 北京+天津
gdp2_hemin_tj  = @coef(1)  + @coef(2) * d(gdp_momn_bj)  + @coef(3) * d_gdp2_hemin_tj

In [9]:
test1 = df_ev[["gdp2_hemin_tj","gdp_momn_bj","d_gdp2_hemin_tj"]]
test1["gdp_momn_bj_diff"] = test1["gdp_momn_bj"] - test1["gdp_momn_bj"].shift(1)
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_hemin_tj ~  gdp_momn_bj_diff + d_gdp2_hemin_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:          gdp2_hemin_tj   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     242.2
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00411
Time:                        01:39:47   Log-Likelihood:                -13.445
No. Observations:                   5   AIC:                             32.89
Df Residuals:                       2   BIC:                             31.72
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
Intercept          304.7006      3.926     77.616      0.000     287.810     321.592
gdp_momn_bj_diff     0.4804      0.031     15.264      0.004       0.345       0.616
d_gdp2_hemin_tj     70.7898      3.669     19.295      0.003      55.004      86.575
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   0.683
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.524
Skew:                           0.749   Prob(JB):                        0.769
Kurtosis:                       2.476   Cond. No.                         205.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq204 : gdp2_nmin_tj 行为方程 ok
gdp2_nmin_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj  + @coef(3) * d_gdp2_nmin_tj


In [10]:
test1 = df_ev[["gdp2_nmin_tj","gdp2_tran_tj","d_gdp2_nmin_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_nmin_tj ~  gdp2_tran_tj + d_gdp2_nmin_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_nmin_tj   R-squared:                       0.996
Model:                            OLS   Adj. R-squared:                  0.992
Method:                 Least Squares   F-statistic:                     235.3
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00423
Time:                        01:41:31   Log-Likelihood:                -17.685
No. Observations:                   5   AIC:                             41.37
Df Residuals:                       2   BIC:                             40.20
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       -183.4866     30.144     -6.087      0.026    -313.186     -53.787
gdp2_tran_tj       0.2700      0.017     16.109      0.004       0.198       0.342
d_gdp2_nmin_tj   104.0133      6.982     14.897      0.004      73.972     134.055
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   2.601
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.368
Skew:                           0.263   Prob(JB):                        0.832
Kurtosis:                       1.778   Cond. No.                     9.22e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq205 : gdp2_birn_tj行为方程 ok
gdp2_birn_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj  + @coef(3) * d_gdp2_birn_tj

In [11]:
test1 = df_ev[["gdp2_birn_tj","gdp2_tran_tj","d_gdp2_birn_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_birn_tj ~  gdp2_tran_tj + d_gdp2_birn_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_birn_tj   R-squared:                       0.998
Model:                            OLS   Adj. R-squared:                  0.996
Method:                 Least Squares   F-statistic:                     563.1
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00177
Time:                        01:42:36   Log-Likelihood:                -16.063
No. Observations:                   5   AIC:                             38.13
Df Residuals:                       2   BIC:                             36.95
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept       -312.3709     22.265    -14.030      0.005    -408.168    -216.574
gdp2_tran_tj       0.3732      0.012     30.122      0.001       0.320       0.427
d_gdp2_birn_tj    55.9863      6.874      8.145      0.015      26.411      85.562
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.548
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.629
Skew:                           0.119   Prob(JB):                        0.730
Kurtosis:                       1.279   Cond. No.                     9.44e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.44e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq206 : gdp2_necin_tj 行为方程 ok
log(gdp2_necin_tj)  = @coef(1)  + @coef(2) * log(gdp2_hemin_tj)  + @coef(3) * d_gdp2_necin_tj


In [12]:
test1 = df_ev[["gdp2_necin_tj","gdp2_hemin_tj","d_gdp2_necin_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " np.log(gdp2_necin_tj) ~  np.log(gdp2_hemin_tj) + d_gdp2_necin_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                              OLS Regression Results                             
=================================================================================
Dep. Variable:     np.log(gdp2_necin_tj)   R-squared:                       0.992
Model:                               OLS   Adj. R-squared:                  0.984
Method:                    Least Squares   F-statistic:                     124.5
Date:                   Tue, 14 Jul 2020   Prob (F-statistic):            0.00797
Time:                           01:44:25   Log-Likelihood:                 5.9772
No. Observations:                      5   AIC:                            -5.954
Df Residuals:                          2   BIC:                            -7.126
Df Model:                              2                                         
Covariance Type:               nonrobust                                         
=========================================================================================
                            coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------
Intercept               -16.0022      1.905     -8.401      0.014     -24.197      -7.807
np.log(gdp2_hemin_tj)     3.2727      0.327     10.016      0.010       1.867       4.679
d_gdp2_necin_tj           0.5964      0.051     11.698      0.007       0.377       0.816
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   3.213
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.270
Skew:                          -0.121   Prob(JB):                        0.874
Kurtosis:                       1.887   Cond. No.                         221.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

## Eq207 : gdp2_nein_tj 行为方程 ok
gdp2_nein_tj  = @coef(1)  + @coef(2) * gdp2_tran_tj


In [13]:
test1 = df_ev[["gdp2_nein_tj","gdp2_tran_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_nein_tj ~  gdp2_tran_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_nein_tj   R-squared:                       0.992
Model:                            OLS   Adj. R-squared:                  0.990
Method:                 Least Squares   F-statistic:                     394.7
Date:                Tue, 14 Jul 2020   Prob (F-statistic):           0.000279
Time:                        01:45:51   Log-Likelihood:                -14.060
No. Observations:                   5   AIC:                             32.12
Df Residuals:                       3   BIC:                             31.34
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
================================================================================
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
Intercept     -137.6475     11.917    -11.551      0.001    -175.572     -99.723
gdp2_tran_tj     0.1316      0.007     19.866      0.000       0.111       0.153
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.700
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.573
Skew:                           0.446   Prob(JB):                        0.751
Kurtosis:                       1.602   Cond. No.                     9.22e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 9.22e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

## Eq208 : gdp2_epin_tj 行为方程 ok
gdp2_epin_tj  = @coef(1)  + @coef(2) * gdp2_hemin_tj  + @coef(3) * d_gdp2_epin_tj

In [14]:
test1 = df_ev[["gdp2_epin_tj","gdp2_hemin_tj","d_gdp2_epin_tj"]]
test1 = test1[(test1.index>2013) & (test1.index <2019)]
results = sm.ols(formula = " gdp2_epin_tj ~  gdp2_hemin_tj + d_gdp2_epin_tj",data = test1 ).fit()
results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:           gdp2_epin_tj   R-squared:                       0.997
Model:                            OLS   Adj. R-squared:                  0.994
Method:                 Least Squares   F-statistic:                     321.8
Date:                Tue, 14 Jul 2020   Prob (F-statistic):            0.00310
Time:                        01:47:25   Log-Likelihood:                -6.2136
No. Observations:                   5   AIC:                             18.43
Df Residuals:                       2   BIC:                             17.26
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==================================================================================
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept        154.7850      3.714     41.676      0.001     138.805     170.765
gdp2_hemin_tj      0.0548      0.011      5.135      0.036       0.009       0.101
d_gdp2_epin_tj    19.9541      0.802     24.879      0.002      16.503      23.405
==============================================================================
Omnibus:                          nan   Durbin-Watson:                   1.116
Prob(Omnibus):                    nan   Jarque-Bera (JB):                0.801
Skew:                           0.972   Prob(JB):                        0.670
Kurtosis:                       2.744   Cond. No.                     2.18e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.18e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""